# FINAL PROJECT: FUNCTIONAL PROGRAMMING

Jorge Parreño Hernández - 100429982

Carlos Suárez Pardo - 100429792


####  Functionalities not covered

In exercise 11, we do not check for duplicate descriptors when event is read.

We do not extend the possibility of repeated events to exercises 9 and 10. After coming very close to not implementing those at all due to difficulties, we did it on exercise 8 but came to the conclusion that (1) their inclusion is not specified in 9 and 10 as it is in 8 and (2) events with a particular frequency ultimately mean several events, which did not make too much sense for the purpose of tasks 9 and 10

### Project summary

The following project consists in a series of function for handling event scheduling in a series of agendas by developing a series of data-types and functions to model that satisfy the requirements with those being.

 Each agenda has an owner, defined as a string, and a list of events included. For each Event we have a list of descriptors, that can be in any order, and some of them may or may not appear, with the different types being:


1. Name: a string describing the event.
2. Category: four possible types, { $personal$, $health$, $work$, $management$}
3. Date: defined with fields Day($Int$), month($String$), year($Int$), workday($Bool$).
4. Initial: initial hour of the event.
5. Final: final hour of the event.
6. Duration: length in minutes of event.
7. Persons: list of persons involved with the owner of the corresponding agenda always being first.
8. Place: location of the event.
9. Frequency: the number of times an event is included, it may be either $punctual$, $weekday$ or $weekly$. If it is not punctual the date parameter indicates the first occurrence of the event.
10. Preferences: stores the time, place and person preferences for the event in question. Earliest time, latest time, location so that it is scheduled when there is another event in that same location and person, to be scheduled before and after an event involving that same person.


As stated before throughout the completion of the 11 tasks, we implemented types to model the requirements specified above.

In [1]:
-- 1: type definition for each type

-- name type
type Name = String 

-- category type
data Category = Personal | Health | Work | Management deriving Eq

-- date type
data Date = Date {day::Int, month::Int, year::Int, workday::Bool} deriving Eq

--time type, to use for initial and final hour types, as well as inside preference type
data Time = Time {hour::Int, minutes::Int} deriving Eq

-- initial type
type Initial = Time

-- final type
type Final = Time

-- duration type
type Duration = Int

-- person type
newtype Persons = Persons [String]

-- place type 
type Place = String

-- frequency type, can only be one of 4, for the last 3 we have a frequency along with a starting date and a
-- number of repetitions, so we have different data sub-types and constructors
data Recurring = Daily | Weekday | Weekly deriving Eq
data Frequency = Punctual | NP {frequency::Recurring, date::Date, repetitions::Int} deriving Eq

-- preferences type
-- earliest starting time, latest ending time, 
-- same place (preference for the event to be scheduled where there's already one scheduled),
-- same person (preference for the event to be scheduled close to an already scheduled one that involves the person)
data Preferences = Preferences {earliest::Time, latest::Time, place::Place, person::String} deriving Eq

-- creating a proper Eq for lists of persons, this will be important for several parts later
instance Eq Persons where
    Persons p1 == Persons p2 = (p1 == p2) && (length p1 == length p2)

### Exercise 1

In this section we are tasked with performing exhaustive type definitions to cover the requirements detailed above. To do so we defined the following types:

1. $Name$ as string
2. $Category$ as one of the four permitted values
3. $Date$ with a constructor with the 4 fields specified
4. $Time$ with fields for hour and minute
5. type $Initial$ which we defined as an instance of $Time$
6. type $Final$ which we also define as an instance of $Time$
7. type $Duration$ simply an integer
8. newtype $Persons$ as a list of strings
9. $Recurring$ which takes 3 values corresponding to the possible types or recurrent events, i.e, Daily, weekday and weekly
10. $Frequency$ which is either punctual or NP (non-punctual) with this second option having a constructor that has three fields, frequency of type $Recurring$ a date, and a number of repetitions.
11. $Preferences$ with a 4-field constructor, accepting the earliest time, which is obviously an instance of $Time$, latest time and person preferences as specified by the statement.
12. $Descriptor$ which can be one of all the previously specified types
13.  $Event$ which is a list of descriptors.
14.  $Agenda$ which has two fields in the constructor, owner, which is of type $Name$ and events which is a list composed of instances of type $Event$.

With all these types we cover all the possibilities that the statement encompasses in the simplest way.

### Exercise 2

The goal of this exercise is to develop a method to print-out an event in a visually pleasant manner following the specified order of descriptors, to achieve it the first thing we did was creating a show instance for the $Descriptor$ type, covering each of the possible types that the descriptor can take, then two print we define the following method to print.

Also, we create methods to check and return a descriptor is present $getXif$, to reorder them in the appropriate fashion.

In [2]:
-- Part 2: function to print event in a neat manner

-- 2.1
-- We must make sure each individual data type is printed neatly
-- We will define an instance Show {data type} for every single data type to achieve it

instance Show Category where
    show Personal = "Category: Personal"
    show Health = "Category: Health"
    show Work = "Category: Work"
    show Management = "Category: Management"

-- itiswkd function turns the Bool value into what we want to print (a string indicating either workday or not a workday)
isitwkd::Bool->String
isitwkd True = "workday"
isitwkd False = "not a workday"

instance Show Date where
    show (Date d m y w) = "Date: " ++ show d ++ "-" ++ show m ++ "-" ++ show y ++ ", " ++ isitwkd w

-- twopositions function helps put numbers in a proper hh:mm format
twopositions::Int->String
twopositions a
    | a<10 = "0" ++ show a
    | otherwise = show a
instance Show Time where 
    show (Time h m) = twopositions h ++ ":" ++twopositions m

instance Show Recurring where
    show Daily = "Frequency: Daily"
    show Weekday = "Frequency: Weekday"
    show Weekly = "Frequency: Weekly"
-- reimplementation of Show Recurring inside of Show Frequency
instance Show Frequency where 
    show Punctual = "Frequency: Punctual"
    show (NP a b c) = show a ++ "; First " ++ show b ++ "; number of repetitions: " ++ show c
    
instance Show Preferences where
    show (Preferences e l pl pr) = "Preferences: Earliest Time: " ++ show e ++ ", Latest Time: " ++ show l ++ ", Place: " ++ pl ++ ", Person: " ++ pr

In [3]:
-- 2.2: type definition for Descriptor 'super-type', Event and Agenda
data Descriptor = DName Name | DCategory Category | DDate Date | DInitial Initial | DFinal Final | DDuration Duration | DPersons Persons | DPlace Place | DFrequency Frequency | DPreferences Preferences deriving Eq

-- we need to implement a function to print a list neatly for DPersons
printlist:: [String]-> String
printlist [a] = a
printlist (x:xs) = x ++ ", " ++ printlist xs
printlist [] = ""

instance Show Persons where
    show (Persons a) = printlist a
    
instance Show Descriptor where
    show (DName a) = "Name: " ++ a
    show (DCategory a) = show a 
    show (DDate a) = show a
    show (DInitial a) = "Initial Time: " ++ show a
    show (DFinal a) = "Final Time: " ++ show a
    show (DDuration a) = "Duration: " ++ show a ++ " minutes"
    show (DPersons a) = "Attendants: " ++ show a
    show (DPlace a) = "Place: " ++ a 
    show (DFrequency a) = show a
    show (DPreferences a) = show a
    
type Event = [Descriptor]
data Agenda = Agenda {owner::Name, events::[Event]}

In [4]:
-- Part 2: function to print event in a neat manner

-- 23
-- We test printing descriptor by descriptor

print Personal
print(Date 12 1 2020 True)
print(Time 18 0)
print Punctual
print(DName "Study")
print(NP Daily (Date 12 1 2020 True) 7)
print(Preferences (Time 18 0) (Time 19 0) "Madrid" "Jorge")

Category: Personal

Date: 12-1-2020, workday

18:00

Frequency: Punctual

Name: Study

Frequency: Daily; First Date: 12-1-2020, workday; number of repetitions: 7

Preferences: Earliest Time: 18:00, Latest Time: 19:00, Place: Madrid, Person: Jorge

In [5]:
-- plethora of auxiliary functions to check whether inside an event a descriptor exists, to get that descriptor
-- will be used extensively throughtout the completion of the project


-- to check that a descriptor is of some type
isPersons::Descriptor->Bool
isPersons (DPersons _) = True
isPersons _ = False

isFinal::Descriptor->Bool
isFinal (DFinal _) = True
isFinal _ = False

isPlace::Descriptor->Bool
isPlace (DPlace _) = True
isPlace _ = False

isDuration::Descriptor->Bool
isDuration (DDuration _) = True
isDuration _ = False

isPreferences::Descriptor->Bool
isPreferences (DPreferences _) = True
isPreferences _ = False

isDate::Descriptor->Bool
isDate (DDate _) = True
isDate _ = False

isInitial::Descriptor->Bool
isInitial (DInitial _) = True
isInitial _ = False

isName::Descriptor->Bool
isName (DName _) = True
isName _ = False

isFrequency::Descriptor->Bool
isFrequency (DFrequency NP {}) = True
isFrequency _ = False

isFreq::Descriptor->Bool
isFreq (DFrequency _) = True
isFreq _ = False

isCategory::Descriptor->Bool
isCategory (DCategory _) = True
isCategory _ = False


-- to check whether a descriptor is present inside an event
isThereName::Event->Bool
isThereName [] = False
isThereName (x:xs)
    | isName x = True
    | otherwise = isThereName xs

isThereCategory::Event->Bool
isThereCategory [] = False
isThereCategory (x:xs) 
    | isCategory x = True
    | otherwise = isThereCategory xs
    
isThereDate::Event->Bool
isThereDate [] = False
isThereDate (x:xs)
    | isDate x = True
    | otherwise = isThereDate xs

isThereInitial::Event->Bool
isThereInitial [] = False
isThereInitial (x:xs)
    | isInitial x = True
    | otherwise = isThereInitial xs

isThereFinal::Event->Bool
isThereFinal [] = False
isThereFinal (x:xs)
    | isFinal x = True
    | otherwise = isThereFinal xs


isTherePreferences::Event->Bool
isTherePreferences [] = False
isTherePreferences (x:xs)
    | isPreferences x = True
    | otherwise = isTherePreferences xs

isTherePlace::Event->Bool
isTherePlace [] = False
isTherePlace (x:xs)
    | isPlace x = True
    | otherwise = isTherePlace xs

isTherePersons::Event->Bool
isTherePersons [] = False
isTherePersons (x:xs)
    | isPersons x = True
    | otherwise = isTherePersons xs

isThereFrequency::Event->Bool
isThereFrequency [] = False
isThereFrequency (x:xs)
    | isFrequency x = True
    | otherwise = isThereFrequency xs
    
isThereDuration::Event->Bool
isThereDuration [] = False
isThereDuration (x:xs)
    | isDuration x = True
    | otherwise = isThereDuration xs

-- to get the descriptor of some type inside an event
---------- *****************************
-- used in 8.9
getName'::Descriptor->String
getName' (DName x) = x
getName' _ = ""

getName::Event->String
getName [] = ""
getName (x:xs) 
    | isName x = getName' x
    | otherwise = getName xs
    
getCategory::Event->String
getCategory [] = ""
getCategory (x:xs)
    | isCategory x = show x
    | otherwise = getCategory xs

getDate'::Descriptor->Date
getDate' (DDate (Date d m y w)) = Date d m y w
getDate' _ = Date 0 0 0 False


getDate::Event->Date
getDate [x] = getDate' x
getDate (x:xs) 
    | isDate x = getDate' x
    | otherwise = getDate xs
    
getPlace'::Descriptor->String
getPlace' (DPlace a) = a
getPlace' _ = []

getPlace::Event->Place
getPlace [x] = getPlace' x
getPlace(x:xs) 
    | isPlace x = getPlace' x
    | otherwise = getPlace xs

getPersons'::Descriptor->[String]
getPersons' (DPersons (Persons a)) = a
getPersons' _ = []

getPersons::Event->[String]
getPersons [x] = getPersons' x
getPersons(x:xs) 
    | isPersons x = getPersons' x
    | otherwise = getPersons xs
    

getFreq'::Descriptor->(Recurring,Date,Int)
getFreq' (DFrequency (NP a b c)) = (a,b,c)
getFreq' _ = (Daily, Date 0 0 0 True ,0)

getFreq::Event->(Recurring,Date,Int)
getFreq [x] = getFreq' x
getFreq(x:xs) 
    | isFrequency x = getFreq' x
    | otherwise = getFreq xs
    

getFinal'::Descriptor->Time
getFinal' (DFinal (Time h m)) = Time h m
getFinal' _ = Time 0 0 -- this case will not matter given the order in which we apply functions,
-- as we first check that there is indeed a final time, but we need to define the function exhaustively

getFinal::Event->Final
getFinal [x] = getFinal' x
getFinal (x:xs) 
    | isFinal x = getFinal' x
    | otherwise = getFinal xs
    
getDuration'::Descriptor->Int
getDuration' (DDuration a) = a
getDuration' _ = 0 -- this case will not matter given that, as stated, we can asume the event to be scheduled will
-- always have a duration

getDuration::Event->Int
getDuration [x] = getDuration' x
getDuration(x:xs) 
    | isDuration x = getDuration' x
    | otherwise = getDuration xs
    
    
getInitial'::Descriptor->Time
getInitial' (DInitial (Time h m)) = Time h m
getInitial' _ = Time 0 0 -- this case will not matter given the order in which we apply functions,
-- as we first check that there is indeed an initial time, but we need to define the function exhaustively

getInitial::Event->Initial
getInitial [x] = getInitial' x
getInitial (x:xs) 
    | isInitial x = getInitial' x
    | otherwise = getInitial xs


getPreferences'::Descriptor->Preferences
getPreferences' (DPreferences a) = a
getPreferences' _ = Preferences (Time 0 0) (Time 0 0) "" "" -- this case will not matter given that, as stated, we can asume the event to be scheduled will
-- always have a preference

getPreferences::Event->Preferences
getPreferences [x] = getPreferences' x
getPreferences(x:xs) 
    | isPreferences x = getPreferences' x
    | otherwise = getPreferences xs

In [6]:
--functions to check if descriptor exists and return it if so
getNameIf::Event->[Descriptor]
getNameIf ev
    | isThereName ev = [DName (getName ev)]
    | otherwise = []

matchCase::String->[Descriptor]
matchCase s 
    | s == "Category: Personal" = [DCategory Personal]
    | s == "Category: Work" = [DCategory Work]
    | s == "Category: Health" = [DCategory Health]
    | s == "Category: Management" = [DCategory Management]
    | otherwise = error "invalid category"
    

matchF::(Recurring,Date,Int)->Frequency
matchF (r,d,i)
    | i == 0 = Punctual
    | otherwise = NP r d i
    
getCategoryIf::Event->[Descriptor]
getCategoryIf ev
    | isThereCategory ev = matchCase $ getCategory ev
    | otherwise = []

getDateIf::Event->[Descriptor]
getDateIf ev
    | isThereDate ev = [DDate (getDate ev)]
    | otherwise = []

getInitialIf::Event->[Descriptor]
getInitialIf ev
    | isThereInitial ev = [DInitial (getInitial ev) ]
    | otherwise = []

getFinalIf::Event->[Descriptor]
getFinalIf ev
    | isThereFinal ev = [DFinal (getFinal ev)]
    | otherwise = []

getDurationIf::Event->[Descriptor]
getDurationIf ev
    | isThereDuration ev = [DDuration (getDuration ev)]
    | otherwise = []   

getPersonsIf::Event->[Descriptor]
getPersonsIf ev
    | isTherePersons ev = [DPersons (Persons (getPersons ev))]
    | otherwise = []

getPlaceIf::Event->[Descriptor]
getPlaceIf ev
    | isTherePlace ev = [DPlace(getPlace ev)]
    | otherwise = []

getFreqIf::Event->[Descriptor]
getFreqIf ev
    | isThereFrequency ev = [DFrequency $ matchF $ getFreq ev]
    | otherwise = []

getPrefIf::Event->[Descriptor]
getPrefIf ev 
    | isTherePreferences ev = [DPreferences(getPreferences ev)]
    | otherwise = []

In [7]:
-- function to reorder list of descriptors to match required orde
reorder::[Descriptor]-> [Descriptor]
reorder l = getNameIf l ++ getCategoryIf l ++ getDateIf l ++ getInitialIf l ++ getFinalIf l ++ getDurationIf l ++ getPersonsIf l ++ getPlaceIf l ++ getFreqIf l ++ getPlaceIf l ++ getFreqIf l ++ getPrefIf l

In [8]:
-- before printing we reorder the descriptor list

prettyPrint'::Event->String
prettyPrint' [x] = show x
prettyPrint' (x:xs) = show x ++ " - " ++ prettyPrint' xs

prettyPrint::Event->String
prettyPrint x = prettyPrint' (reorder x)

In [9]:
-- Part 2: function to print event in a neat manner

-- 2.4
-- Testing the printing functions

myEvent = [ DName "exam", DCategory Personal, DDate (Date 22 10 2020 True), DInitial (Time 18 0), DFinal (Time 19 0), DDuration 60, DPersons(Persons ["Carlos", "Jorge"]), DPlace "Madrid", DFrequency Punctual, DPreferences (Preferences (Time 18 0) (Time 19 0) "Madrid" "Jorge")]
-- getCategory dropWhilemyEvent == "personal"
prettyPrint myEvent


"Name: exam - Category: Personal - Date: 22-10-2020, workday - Initial Time: 18:00 - Final Time: 19:00 - Duration: 60 minutes - Attendants: Carlos, Jorge - Place: Madrid - Place: Madrid - Preferences: Earliest Time: 18:00, Latest Time: 19:00, Place: Madrid, Person: Jorge"

### Exercise 3

In this exercise we seek to create a function that receives a date and a list of vacations and returns whether the date in question is a holiday, weekend day or weekday, knowing that our base date, January $1^{st}$ 2022 was a saturday.

The idea is to first check whether the input date is in the holiday list, for which we define an instance $Eq$ for $Date$, and if so return holiday, this is the simplest case. If it is not within the holiday list we are going to  convert the input date into a numeric scale base on the days elapsed since 01/01/0000, and from that result subtract the numeric equivalent of the base date, and our result will be given depending on the modulus.

In [10]:
-- 3. Create a function isWorkingDate Date [Date] that receives a date and a list of holidays and returns if the day
-- is a weekday or weekend/holiday by considering leap years and the fact that 1st Jan 2022 was a Saturday

-- Idea: convert dates to number, substract and then consider moduli

-- 3.1: create a new date data type for the input, since the date data type in exercise 1 requires a Boolean value
-- for workingDay in the constructor, and that would make the exercise trivil
--data Date' = Date' {day::Int, month::Int, year::Int}
-- creating an instance Eq Date' so we can use `elem` later om:
instance Eq Date where
    Date d1 m1 y1 w1 == Date d2 m2 y2 w2 = d1==d2 && m1 == m2 && y1 == y2

-- 3.2: treatment for leap years. First we ctreate functions for establishing whether a year is leap or not and,
-- based on that, how many leap years we had until the one in the Date
isleap::Int -> Bool
isleap y = y `mod` 4 == 0 && (y `mod` 100 /= 0 || y `mod` 400 == 0)

countLeap'::[Int]->Int
countLeap' [] = 0
countLeap' (x:xs)
  | isleap x = 1 + countLeap' xs
  | otherwise = countLeap' xs
  
countLeap::Int->Int
countLeap x = countLeap' [0,1..x]

-- 3.3: creating a list with the number of days per month in a normal year. We will need to use this. The function
-- scanl1 makes the element on each position of the list cummulative (it adds to it all the preceding ones)
days = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
cd = scanl1 (+) days

-- 3.4: converting dates to numbers
-- add days elapsed in current month + days elapsed in completed months that year + days in previous years
-- and then add an extra day for each leap year that may have occured between 01/01/0000 and input date
-- first case also accounts for the fact that the input date may be a leap year in which february may have passed
-- thus we need to account for an extra day 
dateToNum::Date->Int
dateToNum d1@(Date d m y _) 
    | isleap y && m > 2 = d + cd!!(m-1) + (y-1)*365 + countLeap(y-1) + 1
    | otherwise = d + cd!!(m-1) + (y-1)*365 + countLeap(y-1)
    
-- we know January 1st 2022 was saturday
baseDate = dateToNum (Date 1 1 2022 False)

-- 3.5: actual function we are asked for. If the date is in the list of holidays (hd), it will be so. If not, we
-- use an auxiliary function to subtract the number conversions of the input date and the reference date, then\
-- calculate the modulus to determine whether it is a weekend day.
isWorkingDate::Date->[Date]-> String
isWorkingDate d1@(Date d m y w) hd
    | d1 `elem` hd = "Holiday"
    | otherwise = isWrkd d1

isWrkd::Date->String
isWrkd d1@(Date d m y w)
    | (abs(dateToNum d1 - baseDate) `mod` 7) `elem`[0,1] = "Weekend Day"
    | otherwise = "Weekday"

In [11]:
-- 3.6: testing the function
isWorkingDate (Date 7 1 2022 True) [Date 3 1 2022 False, Date 4 1 2022 False, Date 5 1 2022 False]
isWorkingDate (Date 7 1 2022 False) [Date 3 1 2022 False, Date 4 1 2022 False, Date 5 1 2022 False, Date 7 1 2022 False]
isWorkingDate (Date 8 1 2022 False) [Date 3 1 2022 False, Date 4 1 2022 False, Date 5 1 2022 False]

"Weekday"

"Holiday"

"Weekend Day"

### Exercise 4

In this part we must create a function that accepts an agenda and returns all the events sorted from earliest to latest, but also events that have yet to be scheduled will be shown at the end sorted lexicographically by their name.

Our approach consisted on first constructing functions to determine whether a descriptor is a date, initial time, name, to determine whether there is a date inside the list of descriptors of an event, and a method to do the same for the initial time, we also define methods to return the name, initial time and date of the an input event. Lastly we define a function to check whether the event is scheduled or not, checking that both the date and the initial time exist.

Once we have defined these auxiliary function to extract, and check the type of descriptors. We define $Ord$ instances for Date and time necessary to check and sort events in an Agenda. We also define an $Eq$ instances for $Date$.

Then we implemented bubble sorting, based on foldr and that exploits recursion.

In [12]:
-- 4. Create a listAgenda Agenda function that returns all the Events of an Agenda sorted from earliest to latest
-- events which are noy yet scheduled will be shown at the end sorted by their name in alphabetical order

-- Idea: filter scheduled and not scheduled events,
-- establish an order based on whether they are scheduled or not, 
-- as well as orders inside both scheduled and not scheduled based on either name or date + initial time
-- Finally, use a sorting function with that order as criteria

-- 4.1 we need to treat differently events with and without a date + initial time
-- auxiliary functions to check whether a descriptor is in fact a date or initial time
isDate::Descriptor->Bool
isDate (DDate _) = True
isDate _ = False

isInitial::Descriptor->Bool
isInitial (DInitial _) = True
isInitial _ = False

isName::Descriptor->Bool
isName (DName _) = True
isName _ = False

-- method to determine whether an event is scheduled or not
isThereDate::Event->Bool
isThereDate [] = False
isThereDate (x:xs)
    | isDate x = True
    | otherwise = isThereDate xs

isThereInitial::Event->Bool
isThereInitial [] = False
isThereInitial (x:xs)
    | isInitial x = True
    | otherwise = isThereInitial xs

scheduled::Event->Bool
scheduled [] = False
scheduled xs = isThereDate xs && isThereInitial xs

In [13]:
-- 4.2 after determining whether an event is scheduled or not
-- we need a series of functions to extract dates, times and names in order to establish comparisons

getDate'::Descriptor->Date
getDate' (DDate (Date d m y w)) = Date d m y w
getDate' _ = Date 0 0 0 False

getDate::Event->Date
getDate [x] = getDate' x
getDate (x:xs) 
    | isDate x = getDate' x
    | otherwise = getDate xs
    
getInitial'::Descriptor->Time
getInitial' (DInitial (Time h m)) = Time h m
getInitial' _ = Time 0 0 -- this case will not matter given the order in which we apply functions,
-- as we first check that there is indeed an initial time, but we need to define the function exhaustively

getInitial::Event->Initial
getInitial [x] = getInitial' x
getInitial (x:xs) 
    | isInitial x = getInitial' x
    | otherwise = getInitial xs

-- for not scheduled events
getName::Event->String
getName [] = ""
getName (x:xs) 
    | isName x = show x
    | otherwise = getName xs

In [14]:
-- 4.3 we need functions to compare dates as well as names alphabetically

-- haskell automatically compares strings lexicographically, just like a dictionary

-- We create Ord and Eq instances for Date and Time

instance Ord Date where
    dt1@(Date d1 m1 y1 _) >  dt2@(Date d2 m2 y2 _) = y1>y2 || (y1==y2 && m1>m2) || (y1==y2 && m1==m2 && d1>d2)
    dt1@(Date d1 m1 y1 _) >=  dt2@(Date d2 m2 y2 _) = y1>y2 || (y1==y2 && m1>m2) || (y1==y2 && m1==m2 && d1>=d2)
    dt1@(Date d1 m1 y1 _) <  dt2@(Date d2 m2 y2 _) = y1<y2 || (y1==y2 && m1<m2) || (y1==y2 && m1==m2 && d1<d2)
    dt1@(Date d1 m1 y1 _) <=  dt2@(Date d2 m2 y2 _) = y1<y2 || (y1==y2 && m1<m2) || (y1==y2 && m1==m2 && d1<=d2)

instance Eq Date where
    dt1@(Date d1 m1 y1 _) == dt2@(Date d2 m2 y2 _) = d1 == d2 && m1 == m2 && y1 == y2
    
instance Ord Time where
    t1@(Time h1 m1) < t2@(Time h2 m2) = h1<h2 || (h1==h2 && m1<m2)
    t1@(Time h1 m1) > t2@(Time h2 m2) = h1>h2 || (h1==h2 && m1>m2)
    t1@(Time h1 m1) <= t2@(Time h2 m2) = h1<h2 || (h1==h2 && m1<=m2)
    t1@(Time h1 m1) >= t2@(Time h2 m2) = h1>h2 || (h1==h2 && m1>=m2)

In [15]:
-- 4.4 now all that's left is use the previous functions and criteria to establish a
-- complex enough comparing function for Event and use that function as a criteria in a sorting function
isPrevious::Event->Event->Bool
isPrevious event1 event2 
    | scheduled event1 && not (scheduled event2) = True
    | not (scheduled event1) && scheduled event2 = False
    | scheduled event1 && scheduled event2 && getDate event1 <getDate event2 = True
    | scheduled event1 && scheduled event2 && getDate event1 >getDate event2 = False
    | scheduled event1 && scheduled event2 && getDate event1 ==getDate event2 && getInitial event1 < getInitial event2 = True
    | scheduled event1 && scheduled event2 && getDate event1 ==getDate event2 && getInitial event1 >= getInitial event2 = False
    | not (scheduled event1) && not (scheduled event2) && getName event1 <getName event2 = True
    | not (scheduled event1) && not (scheduled event2) && getName event1 >=getName event2 = False
    | otherwise = False

In [16]:
-- 4.5 final sorting function
listAgenda :: Agenda -> [Event]
listAgenda (Agenda _ agenda) = foldr bubble [] agenda
                where
                bubble x [] = [x]
                bubble x (y:ys) | isPrevious x y = x:y:ys
                                | otherwise = y:bubble x ys

In [17]:
-- 4.6: testing the function
event1 = [DDate (Date 24 10 2020 True), DInitial (Time 18 0), DFinal (Time 19 0)]
event2 = [DDate (Date 19 10 2020 True), DInitial (Time 18 0), DFinal (Time 19 0)]
event3 = [DDate (Date 22 10 2020 True), DInitial (Time 15 0), DFinal (Time 16 0)]
event4 = [DDate (Date 22 10 2020 True), DInitial (Time 18 0), DFinal (Time 19 0)]
event5 = [DName "Carlos", DInitial (Time 18 0), DFinal (Time 19 0)]
event6 = [DName "Antonio", DDate (Date 22 10 2020 True)]
event7 = [DName "Jorge", DDuration 90]

myAgenda = Agenda "Carlos" [event1, event2, event3, event4, event5, event6, event7]

-- functions to print lists of events and agendas (will be useful from now on testing several parts of the project)
printEvents::[Event]->String
printEvents [] = ""
printEvents [x] = prettyPrint x
printEvents (x:xs) = prettyPrint x ++ "                                                                                                                      " ++ printEvents xs   
printAgenda::Agenda->String
printAgenda (Agenda name events) = name ++ " agenda info:                                                                                                             " ++ printEvents events

printAgenda myAgenda
printEvents (listAgenda myAgenda)

"Carlos agenda info:                                                                                                             Date: 24-10-2020, workday - Initial Time: 18:00 - Final Time: 19:00                                                                                                                      Date: 19-10-2020, workday - Initial Time: 18:00 - Final Time: 19:00                                                                                                                      Date: 22-10-2020, workday - Initial Time: 15:00 - Final Time: 16:00                                                                                                                      Date: 22-10-2020, workday - Initial Time: 18:00 - Final Time: 19:00                                                                                                                      Name: Carlos - Initial Time: 18:00 - Final Time: 19:00                                                                             

"Date: 19-10-2020, workday - Initial Time: 18:00 - Final Time: 19:00                                                                                                                      Date: 22-10-2020, workday - Initial Time: 15:00 - Final Time: 16:00                                                                                                                      Date: 22-10-2020, workday - Initial Time: 18:00 - Final Time: 19:00                                                                                                                      Date: 24-10-2020, workday - Initial Time: 18:00 - Final Time: 19:00                                                                                                                      Name: Antonio - Date: 22-10-2020, workday                                                                                                                      Name: Carlos - Initial Time: 18:00 - Final Time: 19:00                                              

### Exercise 5

In this section we create a event insertion function that returns the input agenda with the input event included. The idea in this section is to first insert the input event at the end of the list of the agenda, and then sort it, returning the agenda with the list of the events sorted, using the method defined in the previous exercise.


In [18]:
-- 5 Create an insert Event Agenda function that returns the Agenda with the Event included

-- Idea: insert the Event at the end of the Agenda and then sort the agenda
-- That way (1) we reimplement one of the designed methods and (2) we keep the agenda in order

insert::Event->Agenda->Agenda
insert e (Agenda name events) = Agenda name $ listAgenda $ Agenda name $ events ++ [e]

In [19]:
-- 5.1: testing the function
agenda = Agenda "Carlos" [event3, event4]
newAgenda1 = insert event2 agenda
newAgenda2 = insert event1 newAgenda1
printAgenda agenda
printAgenda newAgenda1
printAgenda newAgenda2

"Carlos agenda info:                                                                                                             Date: 22-10-2020, workday - Initial Time: 15:00 - Final Time: 16:00                                                                                                                      Date: 22-10-2020, workday - Initial Time: 18:00 - Final Time: 19:00"

"Carlos agenda info:                                                                                                             Date: 19-10-2020, workday - Initial Time: 18:00 - Final Time: 19:00                                                                                                                      Date: 22-10-2020, workday - Initial Time: 15:00 - Final Time: 16:00                                                                                                                      Date: 22-10-2020, workday - Initial Time: 18:00 - Final Time: 19:00"

"Carlos agenda info:                                                                                                             Date: 19-10-2020, workday - Initial Time: 18:00 - Final Time: 19:00                                                                                                                      Date: 22-10-2020, workday - Initial Time: 15:00 - Final Time: 16:00                                                                                                                      Date: 22-10-2020, workday - Initial Time: 18:00 - Final Time: 19:00                                                                                                                      Date: 24-10-2020, workday - Initial Time: 18:00 - Final Time: 19:00"

### Exercise 6

This exercise consisted in creating a function to search events that match a list of descriptors within an input agenda.

First a function to check whether a descriptor is contained in an event $containsOneDesc$, then we define another function to check whether an event contains all descriptors $containsAllDesc$ in the input list, using the previously defined function for one descriptor. 

Once we have this two functions we just use list comprehension to iterate through list and fetch the events that match the descriptors.

In [20]:
-- 6 Create a searchEvents [Descriptor] Agenda function that returns all the events of the Agenda with 
-- such list of Descriptors

-- Idea: for each Event in the Agenda: if all the descriptors in the input list are included in the event,
-- return it

containsOneDesc::Descriptor->Event->Bool
containsOneDesc desc event
    | desc `elem` event = True
    | otherwise = False
containsAllDesc::[Descriptor]->Event->Bool
containsAllDesc [x] event = containsOneDesc x event 
containsAllDesc (x:xs) event = containsOneDesc x event && containsAllDesc xs event

-- Implementing the function
searchEvents::[Descriptor]->Agenda->Agenda
searchEvents xs (Agenda n agenda) = Agenda n [a | a<-agenda, containsAllDesc xs a]

In [21]:
-- 6.1 testing the function
event5 = [DCategory Personal, DDate (Date 18 10 2020 True), DDuration 60, DPlace "Madrid"]
event6 = [DCategory Management, DDate (Date 22 10 2020 True), DDuration 60, DPlace "Barcelona"]
event7 = [DCategory Personal, DDate (Date 22 10 2020 True), DPlace "Santiago", DFrequency Punctual]
event8 = [DCategory Management, DDate (Date 22 10 2020 True), DDuration 80, DFrequency Punctual]
agenda2 = Agenda "Carlos" [event5,event6,event7,event8]
descriptors = [DDate (Date 22 10 2020 True), DDuration 60]
printAgenda (searchEvents descriptors agenda2)

"Carlos agenda info:                                                                                                             Category: Management - Date: 22-10-2020, workday - Duration: 60 minutes - Place: Barcelona - Place: Barcelona"

### Exercise 7

In this section we want to delete events that have some of the input descriptors from the input agenda, reusing the $containsAllDesc$ function we do the deletion with a list comprehension based approach.

In [22]:
-- 7 Create a deleteEvents [Descriptor] Agenda function that returns an Agenda where the events having those 
-- descriptors are removed

-- Idea: parallel to the logic in (6)
-- -- 'if the events has those descriptors they are removed' = 'if the events don't have all of those 
-- descriptors they are kept (returned)'

-- Implementing the function
deleteEvents::[Descriptor]->Agenda->Agenda
deleteEvents xs (Agenda n agenda) = Agenda n [a | a<-agenda, not (containsAllDesc xs a)]

In [23]:
-- 7.1 testing the function
descriptors2 = [DDate (Date 22 10 2020 True), DDuration 60]
printAgenda (deleteEvents descriptors2 agenda2)

"Carlos agenda info:                                                                                                             Category: Personal - Date: 18-10-2020, workday - Duration: 60 minutes - Place: Madrid - Place: Madrid                                                                                                                      Category: Personal - Date: 22-10-2020, workday - Place: Santiago - Place: Santiago                                                                                                                      Category: Management - Date: 22-10-2020, workday - Duration: 80 minutes"

### Exercise 8

In this exercise we are tasked with developing a function $scheduleEvent$ $Event$ $Agenda$ $StartDate$ $EndDate$, in which we have to schedule an event in the agenda according to preferences.

We proceed under the assumptions that the input agenda is always sorted and that the input date will always have the duration, either explicitly or we can compute it by subtracting the final and initial times.

The different cases we consider are the following ones:

+  Start date $>$ final date: we raise an error.
+  Duration is directly specify, we use it as criteria to find slots.
+  Duration not specified: we use the time difference between initial and final times as criteria.
+  No preferences specified: we will insert the event in the first slot available of the        appropriate length.
+ Only one preference: call method to return slots that satisfy that preference, if no slots are returned, we treat it as the previous case.
+ Several preferences: we look for the intersection of the results of individual functions tat look for the slots satisfying each individual preferences. If there are none we treat it as the previous case.
+ if the function does not return any slots for the event, then we return the original agenda, as the event cannot be scheduled.


The first thing we do is define auxiliary functions to determine the duration of the event. We already have isInitial and getInitial functions defined in exercise 4 to check if a descriptor is the initial time and return the initial time. We now define isFinal, getFinal, isDuration, getDuration, isThereDuration that returns True if the duration is directly specified in the list of descriptors of an event and False otherwise and findDuration to compute event length if it is not explicitly given.

We then define a slot data type, with fields initial, final, date, persons and place, also we define functions to generate equally spaced slots within a day, and for each event we check whether it is included within that time frame and remove the corresponding slots from the slot list. 

We define a decision tree to check and schedule, depending on whether the event has preferences, which preferences, how they are defined (explicit duration given, or need to compute it), with functions to extract and check data from the input event.


In [24]:
-- 8 Create a scheduleEvent Event Agenda StartDate EndDate that inserts the Event in the Agenda and schedules it
-- as earliest as possible taking into account the preferences if any. If preferences cannot be fulfilled some or all
-- of them will be ignored iteratively. If the vent cannot be scheduled the original idea will be returned. Notice
-- that the event can be partially or fully defined in terms of date.

-- Assumptions (taken from Professors' email)
-- 1. La Agenda estará siempre ordenada según el criterio de la función listAgenda 
-- (de hecho yo siempre ejecutaría esa función sobre cada nueva agenda)
-- 2. La cita a insertar tendrá siempre la duración, bien directamente o bien mediante
-- la hora de inicio y de fin. Puede incluso tener fecha, initial_time y final_tie

-- LOGIC/APPROACH

-- cases:

-- start date is after final date: we will raise an error

-- duration is specified: we will use duration as a criteria for finding slots

-- duration is not specified: we will use the difference between initial and final time as criteria for finding slots

-- there are no preferences: we will insert it in the first slot appropriate for the given duration

-- there is one preference: we design a function to look for that preference and try to return only slots that
-- satisfy it. If there are none, we operate as if there was no preference

-- there are several preferences: we look for the intersection of the results of individual functions that
-- look for the slots satisfying each individual preferences. If there are none, we go for the first preference

-- If the particular function we need returns no slots, we return the original Agenda

-- If the event is repeated with a particular frequency, we need to create a function that gives us all the 
-- events that we will actually need to cover within the start date and final date given that frequency and
-- do all the previous for all of them

In [25]:
-- 8.1 auxiliary functions to determine the duration of the event

-- we already have isInitial and getInitial
-- we need isFinal and getFinal, isThereDuration, isDuration and getDuration
isFinal::Descriptor->Bool
isFinal (DFinal _) = True
isFinal _ = False

getFinal'::Descriptor->Time
getFinal' (DFinal (Time h m)) = Time h m
getFinal' _ = Time 0 0 -- this case will not matter given the order in which we apply functions,
-- as we first check that there is indeed a final time, but we need to define the function exhaustively

getFinal::Event->Final
getFinal [x] = getFinal' x
getFinal (x:xs) 
    | isFinal x = getFinal' x
    | otherwise = getFinal xs

isDuration::Descriptor->Bool
isDuration (DDuration _) = True
isDuration _ = False

isThereDuration::Event->Bool
isThereDuration [] = False
isThereDuration (x:xs)
    | isDuration x = True
    | otherwise = isThereDuration xs

getDuration'::Descriptor->Int
getDuration' (DDuration a) = a
getDuration' _ = 0 -- this case will not matter given that, as stated, we can asume the event to be scheduled will
-- always have a duration

getDuration::Event->Int
getDuration [x] = getDuration' x
getDuration(x:xs) 
    | isDuration x = getDuration' x
    | otherwise = getDuration xs
    
-- when no duration, must calculate difference between final and initial times

diff::Time->Time->Int
diff (Time h1 m1) (Time h2 m2) = h1*60 + m1 - h2*60 - m2

findDuration::Event->Int
findDuration event
    | isThereDuration event = getDuration event
    | otherwise = diff (getFinal event) (getInitial event)

In [26]:
-- 8.2 
-- we need a slot data type
-- first, we will structure a standard day in slots of time for the given duration which,
-- from 8.1, we either know or infer from initial and final time
data Slot = Slot {initial::Time, final::Time, date::Date, persons::Persons, place::Place}
instance Show Slot where
    show (Slot ini fin date persons place) = show ini ++ "-" ++ show fin ++ " | " ++ show date ++ " | Persons: " ++ show persons ++ " | Place: " ++ show place ++ "                                                                "
    
getTotalSlots1::Duration->[Int]
getTotalSlots1 d = [0,d..1440]

numbertoHour::Int->Time
numbertoHour n = Time (n `div` 60) (n - 60*(n `div` 60))
    
getTotalSlots2::[Int]->[Time]
getTotalSlots2 = map numbertoHour

getTotalSlots3::[Time]->[(Time,Time)]
getTotalSlots3 slots2 = zip slots2 $ tail slots2

getTotalSlots::Duration->[(Time,Time)]
getTotalSlots d = getTotalSlots3 $ getTotalSlots2 $ getTotalSlots1 d
-- testing
getTotalSlots 40

[(00:00,00:40),(00:40,01:20),(01:20,02:00),(02:00,02:40),(02:40,03:20),(03:20,04:00),(04:00,04:40),(04:40,05:20),(05:20,06:00),(06:00,06:40),(06:40,07:20),(07:20,08:00),(08:00,08:40),(08:40,09:20),(09:20,10:00),(10:00,10:40),(10:40,11:20),(11:20,12:00),(12:00,12:40),(12:40,13:20),(13:20,14:00),(14:00,14:40),(14:40,15:20),(15:20,16:00),(16:00,16:40),(16:40,17:20),(17:20,18:00),(18:00,18:40),(18:40,19:20),(19:20,20:00),(20:00,20:40),(20:40,21:20),(21:20,22:00),(22:00,22:40),(22:40,23:20),(23:20,24:00)]

In [27]:
-- 8.3
-- now we have a vector of hour pairs for a standard day structured for a particular duration
-- in order to get all the existing slots, we must make one vector of slots with all the combinations
-- of initial and final times from these vector and dates between date1 and date2
-- person and place will be both initialized to "" (empty) for all slots

isWorkingBool::String->Bool
isWorkingBool "Weekend Day" = False
isWorkingBool "Holiday" = False
isWorkingBool _ = True

nextDate::Date->Date
nextDate (Date d m y w)
    | d==30 && (m==4 || m==6 || m==9 || m==11) = Date 1 (m+1) y (isWorkingBool(isWorkingDate (Date 1 (m+1) y True) []))
    | d==31 && (m==1 || m==3 || m==5 || m==7 || m==8 || m==10) = Date 1 (m+1) y (isWorkingBool(isWorkingDate (Date 1 (m+1) y True) []))
    | d==31 && m==12 = Date 1 1 (y+1) (isWorkingBool(isWorkingDate (Date 1 1 (y+1) True) []))
    | d==28 && m==2 && not (isleap y) = Date 1 (m+1) y (isWorkingBool(isWorkingDate (Date 1 (m+1) y True) []))
    | d==29 && m==2 && isleap y = Date 1 (m+1) y (isWorkingBool(isWorkingDate (Date 1 (m+1) y True) []))
    | otherwise = Date (d+1) m y (isWorkingBool(isWorkingDate (Date (d+1) m y True) []))

getListDates::Date->Date->[Date]
getListDates date1 date2 = getListDates' [date1] (dateToNum date2 - dateToNum date1 +1)

getListDates'::[Date]->Int->[Date]
getListDates' ys n
    | n==1 = [head ys]
    | length ys == (n-1) = ys ++ [nextDate (last ys)]
    | otherwise = getListDates' (ys ++ [nextDate (last ys)]) n 

In [28]:
-- 8.4
-- that above is the vector with dates
-- with getTotalSlots we had the vector of timestamps
-- now create a new vector of Slots  with all combinations of both vectors

combos::[(Time,Time)]->[Date]->[Slot]
combos xs ys = [Slot a b c (Persons[]) "" | (a,b)<-xs, c<-ys ]

existingSlots::Event->Date->Date->[Slot]
existingSlots event date1 date2 = combos (getTotalSlots (findDuration event)) (getListDates date1 date2)

In this first approach we generate slots of equal length and if the events do not end exactly at the beginning of a slot, we discard what's left of the slot that may be partially occupied.

This is an issue which we solve further on.

In [29]:
-- 8.5
-- now that we have this, we need to substract all slots that are included within the time frame 
-- occupied by all scheduled events, in order to obtain the available events

isIncluded::Slot->Slot->Bool
isIncluded (Slot ini1 fin1 date1 _ _ ) (Slot ini2 fin2 date2 _ _)
    | date1 == date2 && (fin1 <= ini2) = False
    | date1 == date2 && (ini1 >= fin2) = False
    | date1 /= date2 = False
    | otherwise = True

removeOccupied::[Slot]->[Slot]->[Slot]
removeOccupied [x] ys = removeOccupied' x ys
removeOccupied (x:xs) ys = removeOccupied xs (removeOccupied' x ys)

removeOccupied'::Slot->[Slot]->[Slot]
removeOccupied' x [y] = removeOccupied'' x y
removeOccupied' x (y:ys) = removeOccupied'' x y ++ removeOccupied' x ys

removeOccupied''::Slot->Slot->[Slot]
removeOccupied'' x y 
    | isIncluded y x = []
    | otherwise = [y]
    
getSlots::[Event]->[Slot]
getSlots [x]
    -- scheduled
    | isThereInitial x = [Slot (getInitial x) (getFinal x) (getDate x) (Persons[]) ""]
    -- not scheduled: does not affect us
    | otherwise = []
getSlots (x:xs)
    | isThereInitial x = Slot (getInitial x) (getFinal x) (getDate x) (Persons[]) "" : getSlots xs
    | otherwise = getSlots xs
    
availableSlots::Event->Agenda->Date->Date->[Slot]
availableSlots event (Agenda _ xs) date1 date2 = removeOccupied (getSlots xs) (existingSlots event date1 date2)

-- testing
availableSlots [DDuration 60] (Agenda "Carlos" [[DDate (Date 1 1 2022 False), DInitial (Time 12 0), DFinal (Time 14 30)], [DDate (Date 6 1 2022 True), DInitial (Time 16 0), DFinal (Time 16 30)]]) (Date 1 1 2022 False) (Date 9 1 2022 True)
-- it works

[00:00-01:00 | Date: 1-1-2022, not a workday | Persons:  | Place: ""                                                                ,00:00-01:00 | Date: 2-1-2022, not a workday | Persons:  | Place: ""                                                                ,00:00-01:00 | Date: 3-1-2022, workday | Persons:  | Place: ""                                                                ,00:00-01:00 | Date: 4-1-2022, workday | Persons:  | Place: ""                                                                ,00:00-01:00 | Date: 5-1-2022, workday | Persons:  | Place: ""                                                                ,00:00-01:00 | Date: 6-1-2022, workday | Persons:  | Place: ""                                                                ,00:00-01:00 | Date: 7-1-2022, workday | Persons:  | Place: ""                                                                ,00:00-01:00 | Date: 8-1-2022, not a workday | Persons:  | Place: ""                                      

In [30]:
-- 8.6.1
-- now that we have a list with all available slots there are several routes to go

-- events without preference --

-- totally defined
-- if the event has both a date and a timestamp we determine whether it is included in one of available_slots
--selectSlot::Event->Agenda->Date->Date->Slot
--selectSlot event (Agenda _ events) date1 date2 
 --   | isThereDate event && isThereInitial event = selectSlot1 event (Agenda _ events) date1 date2 
    
timesToSlot::Event->Slot
timesToSlot event = Slot (getInitial event) (getFinal event) (getDate event) (Persons[]) ""

anyIncluded::Slot->[Slot]->Bool
anyIncluded x [y] = isIncluded x y
anyIncluded x (y:ys) = isIncluded x y || anyIncluded x ys

selectSlot1::Event->Agenda->Date->Date->Agenda
selectSlot1 event (Agenda name events) date1 date2
    | anyIncluded (timesToSlot event) (getSlots events) = Agenda name events
    | otherwise = insert event (Agenda name events)

-- partially defined
-- if the event has only a date and duration we return the first slot in available_slots that has that date

slotToInitial::Slot->Descriptor
slotToInitial (Slot ini fin date _ _) = DInitial ini
slotToFinal::Slot->Descriptor
slotToFinal (Slot ini fin date _ _) = DFinal fin

selectSlot2::Event->Agenda->Date->Date->Agenda
selectSlot2 event (Agenda name events) date1 date2
    | null (availableSlots event (Agenda name events) (getDate event) (getDate event)) = Agenda name events
    | otherwise = insert (event ++ [slotToInitial (head(availableSlots event (Agenda name events) (getDate event) (getDate event)))] ++ [slotToFinal (head(availableSlots event (Agenda name events) (getDate event) (getDate event)))]) (Agenda name events)

-- undefined
-- if the event does not have a date (thus no timestamp either) we return the first slot in available_slots

slotToDate::Slot->Descriptor
slotToDate (Slot ini fin date _ _) = DDate date

selectSlot3::Event->Agenda->Date->Date->Agenda
selectSlot3 event (Agenda name events) date1 date2
    | null (availableSlots event (Agenda name events) date1 date2) = Agenda name events
    | otherwise = insert (event ++ [slotToInitial (head(availableSlots event (Agenda name events) date1 date2))] ++ [slotToFinal (head(availableSlots event (Agenda name events) date1 date2))] ++ [slotToDate (head(availableSlots event (Agenda name events) date1 date2))]) (Agenda name events)

In [31]:
-- 8.6.2
-- now that we have a list with all available slots there are several routes to go

-- events with a time preference -- ** other preferences are treated starting at 8.7 **

-- obviously a totally defined event will not have preferences to be considered, and it does not make sense
-- for an undefined event to have a time preference, since it doesn't even have a date, so we consider:
-- partially defined: the event has a date and a time prefence
-- we first try to return a slot satisfying the time preferece
-- it that is not possibele, we will ignore the preference and return the earliest slot
-- if that is also impossible (no slot available), we will return the same agenda

isPreferences::Descriptor->Bool
isPreferences (DPreferences _) = True
isPreferences _ = False

isTherePreferences::Event->Bool
isTherePreferences [] = False
isTherePreferences (x:xs)
    | isPreferences x = True
    | otherwise = isTherePreferences xs

getPreferences'::Descriptor->Preferences
getPreferences' (DPreferences a) = a
getPreferences' _ = Preferences (Time 0 0) (Time 0 0) "" "" -- this case will not matter given that, as stated, we can asume the event to be scheduled will
-- always have a preference

getPreferences::Event->Preferences
getPreferences [x] = getPreferences' x
getPreferences(x:xs) 
    | isPreferences x = getPreferences' x
    | otherwise = getPreferences xs
    
initialPref::Preferences->Time
initialPref (Preferences a b c d) = a
finalPref::Preferences->Time
finalPref (Preferences a b c d) = b

sumMin::Time->Duration->Time
sumMin (Time h m) d
    | h + div (m+d) 60 >= 24  = Time 24 0
    | m+d<60 = Time h (m+d)
    | otherwise = Time (h + div (m+d) 60) (m+d-60*div(m+d)60)

selectSlot4::Event->Agenda->Date->Date->Agenda
selectSlot4 event (Agenda name events) date1 date2
    | null (availableSlots event (Agenda name events) (getDate event) (getDate event)) = Agenda name events
    | anyIncluded (timesToSlot (event ++ [DInitial (initialPref(getPreferences event))] ++ [DFinal (sumMin (initialPref(getPreferences event))(getDuration event)) ])) (getSlots events) = insert (event ++ [slotToInitial (head(availableSlots event (Agenda name events) (getDate event) (getDate event)))] ++ [slotToFinal (head(availableSlots event (Agenda name events) (getDate event) (getDate event)))]) (Agenda name events)
    | otherwise = insert (event ++ [DInitial (initialPref(getPreferences event))] ++ [DFinal (sumMin (initialPref(getPreferences event))(getDuration event)) ]) (Agenda name events)

In [32]:
-- 8.7
-- new approach to deal with events that do not occupy exactly one of the predefined slots
-- will also help with preferences other than time preferences
-- will also help with exercise 9

-- in a day to day basis we have a vector with every minute of the day
minutesInDay = [1,2..(24*60-1)]

-- the goal here is, given the minutes available in a day and the
-- the events already scheduled that tday with their respective 
-- initial and final times, be able to get the minutes, in absolute number over
-- all minutes in a day, for which a new free slot starts or ends (key times)

timeToMinute::Time->Int
timeToMinute (Time a b) = 60*a+b

eventToMinute::Event->[Int]
eventToMinute event = [timeToMinute (getInitial event),timeToMinute (getInitial event) +1.. timeToMinute (getFinal event)]

removeMinutes::[Int]->[Int]->[Int]
removeMinutes [x] ys = [x]
removeMinutes (x:xs) ys
    | x `elem` ys = removeMinutes xs ys
    | otherwise = x : removeMinutes xs ys

removeOccupied2::[Event]->[Int]->[Int]
removeOccupied2 [x] ys = removeMinutes ys (eventToMinute x)
removeOccupied2 (x:xs) ys = removeOccupied2 xs (removeMinutes ys (eventToMinute x))
removeOccupied2 [] ys = ys

getKeyTimes'::[Int]->[Int]
getKeyTimes' [x] = [x]
getKeyTimes'(x:xs)
    | head xs == x+1 = getKeyTimes' xs
    | otherwise = x:head xs:getKeyTimes' xs
    
getKeyTimes::[Int]->[Int]
getKeyTimes xs = head xs : getKeyTimes' xs

-- testing
event1 = [DInitial (Time 18 0),  DFinal (Time 19 30), DDate (Date 3 12 2020 True), DPersons(Persons ["Jorge","Gonzalo"]), DPlace "Leganes"]
event2 = [DInitial (Time 5 0), DFinal (Time 8 30), DDate (Date 3 12 2020 True), DPersons(Persons ["Paco","Eugenia"]), DPlace "Madrid"]
events = [event1, event2]
getKeyTimes (removeOccupied2 events minutesInDay)

[1,299,511,1079,1171,1439]

In [33]:
-- 8.8
-- continuing the new approach
-- we want to go from the vector of key times to a list of pairs of times that
-- will represent the free slots

-- approach: in a day to day basis (later we will use a higher level function to iterate through the dates
-- between date1 and date2), for each free slot, if the time available in that slot is greater than or equal
-- to the duration of the event we want to schedule, we will get the corresponding duration at the beginning
-- and end of the slot (that is, right after the previously scheduled 
-- event and right before the event coming afterwards) as our potential time frames for scheduling the event

getInitialMinutes::[Int]->[Int]
getInitialMinutes xs = [(xs!!a)-1 | a<-[0..length xs - 1], even a]
getFinalMinutes::[Int]->[Int]
getFinalMinutes xs = [(xs!!a)+1 | a<-[0..length xs - 1], odd a]
    
getavailableSlots2::[Int]->[Int]->Date->Duration->[Slot]
getavailableSlots2 [x] [y] date duration
    | y - x > duration  = [Slot (Time (div x 60) (x - 60*div x 60)) (Time (div (x+duration) 60) ((x+duration) - 60*div (x+duration) 60)) date (Persons[]) "",Slot (Time (div (y-duration) 60) ((y-duration) - 60*div (y-duration) 60)) (Time (div y 60) (y - 60*div y 60)) date (Persons[]) ""] 
    | y - x == duration = [Slot (Time (div x 60) (x - 60*div x 60)) (Time (div (x+duration) 60) ((x+duration) - 60*div (x+duration) 60)) date (Persons[]) ""]
    | otherwise = []
getavailableSlots2 (x:xs) (y:ys) date duration 
    | y - x > duration  = [Slot (Time (div x 60) (x - 60*div x 60)) (Time (div (x+duration) 60) ((x+duration) - 60*div (x+duration) 60)) date (Persons[]) "",Slot (Time (div (y-duration) 60) ((y-duration) - 60*div (y-duration) 60)) (Time (div y 60) (y - 60*div y 60)) date (Persons[]) ""] ++ getavailableSlots2 xs ys date duration  
    | y - x == duration = Slot (Time (div x 60) (x - 60*div x 60)) (Time (div (x+duration) 60) ((x+duration) - 60*div (x+duration) 60)) date (Persons[]) "" : getavailableSlots2 xs ys date duration 
    | otherwise = getavailableSlots2 xs ys date duration 
    
-- test
getInitialMinutes (getKeyTimes (removeOccupied2 events minutesInDay))
getFinalMinutes (getKeyTimes (removeOccupied2 events minutesInDay))
getavailableSlots2 (getInitialMinutes (getKeyTimes (removeOccupied2 events minutesInDay))) (getFinalMinutes (getKeyTimes (removeOccupied2 events minutesInDay))) (Date 3 12 2020 True) 50

[0,510,1170]

[300,1080,1440]

[00:00-00:50 | Date: 3-12-2020, workday | Persons:  | Place: ""                                                                ,04:10-05:00 | Date: 3-12-2020, workday | Persons:  | Place: ""                                                                ,08:30-09:20 | Date: 3-12-2020, workday | Persons:  | Place: ""                                                                ,17:10-18:00 | Date: 3-12-2020, workday | Persons:  | Place: ""                                                                ,19:30-20:20 | Date: 3-12-2020, workday | Persons:  | Place: ""                                                                ,23:10-24:00 | Date: 3-12-2020, workday | Persons:  | Place: ""                                                                ]

In [34]:
-- 8.9
-- on top of that, since the slot data type has preference values, we will save in those the preferences
-- of events scheduled next to the slot, if any, in order to help schedule events with preference

-- auxiliary functions

eventsInDate::[Event]->Date->[Event]
eventsInDate [x] date
    | getDate x == date = [x]
    | otherwise = []
eventsInDate (x:xs) date
    | getDate x == date = x : eventsInDate xs date
    | otherwise = eventsInDate xs date

isPersons::Descriptor->Bool
isPersons (DPersons _) = True
isPersons _ = False

isTherePersons::Event->Bool
isTherePersons [] = False
isTherePersons (x:xs)
    | isPersons x = True
    | otherwise = isTherePersons xs

getPersons'::Descriptor->[String]
getPersons' (DPersons (Persons a)) = a
getPersons' _ = []

getPersons::Event->[String]
getPersons [x] = getPersons' x
getPersons(x:xs) 
    | isPersons x = getPersons' x
    | otherwise = getPersons xs
    
isPlace::Descriptor->Bool
isPlace (DPlace _) = True
isPlace _ = False

isTherePlace::Event->Bool
isTherePlace [] = False
isTherePlace (x:xs)
    | isPlace x = True
    | otherwise = isTherePlace xs

getPlace'::Descriptor->String
getPlace' (DPlace a) = a
getPlace' _ = []

getPlace::Event->Place
getPlace [x] = getPlace' x
getPlace(x:xs) 
    | isPlace x = getPlace' x
    | otherwise = getPlace xs
    
-- procedure

instance Eq Time where
    Time h1 m1 == Time h2 m2 = h1==h2 && m1==m2

assignSlotPref::Slot->[Event]->Slot
assignSlotPref (Slot a b c d e) [x]
    | getDate x == c && (getInitial x == b || getFinal x == a) && isTherePersons x && isTherePlace x = Slot a b c (Persons (getPersons x)) (getPlace x)
    | getDate x == c && (getInitial x == b || getFinal x == a) && isTherePersons x = Slot a b c (Persons (getPersons x)) e
    | getDate x == c && (getInitial x == b || getFinal x == a) && isTherePlace x = Slot a b c d (getPlace x)
    | otherwise = Slot a b c d e
assignSlotPref (Slot a b c d e) (x:xs)
    | getDate x == c && (getInitial x == b || getFinal x == a) && isTherePersons x && isTherePlace x = Slot a b c (Persons (getPersons x)) (getPlace x)
    | getDate x == c && (getInitial x == b || getFinal x == a) && isTherePersons x = Slot a b c (Persons (getPersons x)) e
    | getDate x == c && (getInitial x == b || getFinal x == a) && isTherePlace x = Slot a b c d (getPlace x)
    | otherwise = assignSlotPref (Slot a b c d e) xs
assignSlotPref (Slot a b c d e) [] = Slot a b c d e
    
assignSlotsPref::[Slot]->[Event]->[Slot]
assignSlotsPref [x] ys = [assignSlotPref x ys]
assignSlotsPref (x:xs) ys = assignSlotPref x ys : assignSlotsPref xs ys
assignSlotsPref [] ys = []

-- testing

myslots = getavailableSlots2 (getInitialMinutes (getKeyTimes (removeOccupied2 events minutesInDay))) (getFinalMinutes (getKeyTimes (removeOccupied2 events minutesInDay))) (Date 3 12 2020 True) 50
myslots
events
assignSlotPref (head myslots) events
assignSlotsPref myslots events

[00:00-00:50 | Date: 3-12-2020, workday | Persons:  | Place: ""                                                                ,04:10-05:00 | Date: 3-12-2020, workday | Persons:  | Place: ""                                                                ,08:30-09:20 | Date: 3-12-2020, workday | Persons:  | Place: ""                                                                ,17:10-18:00 | Date: 3-12-2020, workday | Persons:  | Place: ""                                                                ,19:30-20:20 | Date: 3-12-2020, workday | Persons:  | Place: ""                                                                ,23:10-24:00 | Date: 3-12-2020, workday | Persons:  | Place: ""                                                                ]

[[Initial Time: 18:00,Final Time: 19:30,Date: 3-12-2020, workday,Attendants: Jorge, Gonzalo,Place: Leganes],[Initial Time: 05:00,Final Time: 08:30,Date: 3-12-2020, workday,Attendants: Paco, Eugenia,Place: Madrid]]

00:00-00:50 | Date: 3-12-2020, workday | Persons:  | Place: ""

[00:00-00:50 | Date: 3-12-2020, workday | Persons:  | Place: ""                                                                ,04:10-05:00 | Date: 3-12-2020, workday | Persons: Paco, Eugenia | Place: "Madrid"                                                                ,08:30-09:20 | Date: 3-12-2020, workday | Persons: Paco, Eugenia | Place: "Madrid"                                                                ,17:10-18:00 | Date: 3-12-2020, workday | Persons: Jorge, Gonzalo | Place: "Leganes"                                                                ,19:30-20:20 | Date: 3-12-2020, workday | Persons: Jorge, Gonzalo | Place: "Leganes"                                                                ,23:10-24:00 | Date: 3-12-2020, workday | Persons:  | Place: ""                                                                ]

In [35]:
-- 8.10
-- final steps before the main function
-- definition of functions that determine to path to follow within the decision tree
-- depending on whether the event has preferences, which preferences, how defined it is in terms of date, etc
-- we will call different functions

-- this will allow us to complete every single functionality of 8 except the events with repetition, as well as 9

prefToPlace::Preferences->Place
prefToPlace (Preferences a b c d) = c

prefToPerson::Preferences->String
prefToPerson (Preferences a b c d) = d

getPlacePref::Event->Place
getPlacePref event = prefToPlace (getPreferences event)

getPersonsPref::Event->String
getPersonsPref event = prefToPerson (getPreferences event)

getSlotsWPref::[Event]->Date->Duration->[Slot]
getSlotsWPref events date duration = assignSlotsPref (getavailableSlots2 (getInitialMinutes (getKeyTimes (removeOccupied2 events minutesInDay))) (getFinalMinutes (getKeyTimes (removeOccupied2 events minutesInDay))) date duration) events

availableThroughDates::Duration->[Event]->[Date]->[Slot]
availableThroughDates duration events [x] = getSlotsWPref (eventsInDate events x) x duration
availableThroughDates duration events (x:xs) = getSlotsWPref (eventsInDate events x) x duration ++ availableThroughDates duration events xs

-- testing
availableThroughDates 45 events (getListDates (Date 3 12 2020 True) (Date 4 12 2020 True))
getSlotsWPref events (Date 3 12 2020 True) 45
-- it works (:D)

[00:00-00:45 | Date: 3-12-2020, workday | Persons:  | Place: ""                                                                ,04:15-05:00 | Date: 3-12-2020, workday | Persons: Paco, Eugenia | Place: "Madrid"                                                                ,08:30-09:15 | Date: 3-12-2020, workday | Persons: Paco, Eugenia | Place: "Madrid"                                                                ,17:15-18:00 | Date: 3-12-2020, workday | Persons: Jorge, Gonzalo | Place: "Leganes"                                                                ,19:30-20:15 | Date: 3-12-2020, workday | Persons: Jorge, Gonzalo | Place: "Leganes"                                                                ,23:15-24:00 | Date: 3-12-2020, workday | Persons:  | Place: ""                                                                ,00:00-00:45 | Date: 4-12-2020, not a workday | Persons:  | Place: ""                                                                ,23:15-24:00 | Date: 4-12

[00:00-00:45 | Date: 3-12-2020, workday | Persons:  | Place: ""                                                                ,04:15-05:00 | Date: 3-12-2020, workday | Persons: Paco, Eugenia | Place: "Madrid"                                                                ,08:30-09:15 | Date: 3-12-2020, workday | Persons: Paco, Eugenia | Place: "Madrid"                                                                ,17:15-18:00 | Date: 3-12-2020, workday | Persons: Jorge, Gonzalo | Place: "Leganes"                                                                ,19:30-20:15 | Date: 3-12-2020, workday | Persons: Jorge, Gonzalo | Place: "Leganes"                                                                ,23:15-24:00 | Date: 3-12-2020, workday | Persons:  | Place: ""                                                                ]

In [36]:
-- 8.11 extending our approach to fit events with a frequency

-- approach

-- treat events with punctual frequency like events without a frequency descriptor

-- 1 create a list of dates between date1 and date2 and delete all the dates where
-- the event is not being repeated

-- geting the elements of frequency we need

isFrequency::Descriptor->Bool
isFrequency (DFrequency NP {}) = True
isFrequency _ = False

isThereFrequency::Event->Bool
isThereFrequency [] = False
isThereFrequency (x:xs)
    | isFrequency x = True
    | otherwise = isThereFrequency xs
    
getFreq'::Descriptor->(Recurring,Date,Int)
getFreq' (DFrequency (NP a b c)) = (a,b,c)
getFreq' _ = (Daily, Date 0 0 0 True ,0)

get1st::(Recurring,Date,Int)->Recurring
get1st (a,b,c) = a
get2nd::(Recurring,Date,Int)->Date
get2nd (a,b,c) = b
get3rd::(Recurring,Date,Int)->Int
get3rd (a,b,c) = c

getFreq::Event->(Recurring,Date,Int)
getFreq [x] = getFreq' x
getFreq(x:xs) 
    | isFrequency x = getFreq' x
    | otherwise = getFreq xs
    
-- removing elements from the dates between the date of the event and date2 depending on whether the frequency
-- is weekly or weekday

removeWeekend::[Date]->[Date]
removeWeekend [x] 
    | isWrkd x == "Weekday" = [x]
    | otherwise = []
removeWeekend (x:xs)
    | isWrkd x == "Weekday" = x : removeWeekend xs
    | otherwise = removeWeekend xs
    
removeNotWeekly::[Date]->Date->[Date]
removeNotWeekly [x] y
    | mod ( dateToNum x - dateToNum y ) 7 == 0 = [x]
    | otherwise = []
removeNotWeekly (x:xs) y
    | mod ( dateToNum x - dateToNum y ) 7 == 0 = x : removeNotWeekly xs y
    | otherwise = removeNotWeekly xs y

freqListOfDates::Event->Date->[Date]
freqListOfDates event date2 
    | get1st (getFreq event) == Daily = take (get3rd $ getFreq event) (getListDates (get2nd (getFreq event)) date2)
    | get1st (getFreq event) == Weekly = take (get3rd $ getFreq event) (removeNotWeekly (getListDates (get2nd (getFreq event)) date2) (head(getListDates (get2nd (getFreq event)) date2)))
    | get1st (getFreq event) == Weekday = take (get3rd $ getFreq event) (removeWeekend(getListDates (get2nd (getFreq event)) date2))

-- 2 testing
mydatelist = freqListOfDates [DDate (Date 3 12 2020 True), DInitial (Time 12 0), DFinal (Time 14 0), DFrequency (NP Weekday (Date 4 12 2020 True) 12)] (Date 30 12 2020 True)
print mydatelist

-- 3 we need a new event with the same characteristics of the input event
-- but a new date for every date on the list

dropDate::Event->Event
dropDate [x]
    | isDate x = []
    | otherwise = [x]
dropDate (x:xs)
    | isDate x = dropDate xs
    | otherwise = x : dropDate xs

changeDate::Event->Date->Event
changeDate event date 
    | not (isThereDate event) = event ++ [DDate date]
    | otherwise = dropDate event ++ [DDate date]

createFreqEvents::Event->Date->Date->[Event]
createFreqEvents event date1 date2 = [ changeDate event a | a<- freqListOfDates event date2]

-- 4 testing
date1 = Date 1 1 2020 True
date2 = Date 30 1 2020 True
-- event = [DName "Facetime every weekday", DDate (Date 3 1 2020 True), DInitial (Time 14 0), DFinal (Time 15 0), DFrequency (NP Daily (Date 3 1 2020 True) 5)]
event = [DName "Facetime every day", DDate (Date 3 1 2020 True), DInitial (Time 14 0), DFinal (Time 15 0), DFrequency (NP Weekday (Date 3 1 2020 True) 5)]
printEvents (createFreqEvents event date1 date2)

[Date: 6-12-2020, workday,Date: 7-12-2020, workday,Date: 8-12-2020, workday,Date: 9-12-2020, workday,Date: 10-12-2020, workday,Date: 13-12-2020, workday,Date: 14-12-2020, workday,Date: 15-12-2020, workday,Date: 16-12-2020, workday,Date: 17-12-2020, workday,Date: 20-12-2020, workday,Date: 21-12-2020, workday]

"Name: Facetime every day - Date: 5-1-2020, workday - Initial Time: 14:00 - Final Time: 15:00 - Frequency: Weekday; First Date: 3-1-2020, workday; number of repetitions: 5 - Frequency: Weekday; First Date: 3-1-2020, workday; number of repetitions: 5                                                                                                                      Name: Facetime every day - Date: 6-1-2020, workday - Initial Time: 14:00 - Final Time: 15:00 - Frequency: Weekday; First Date: 3-1-2020, workday; number of repetitions: 5 - Frequency: Weekday; First Date: 3-1-2020, workday; number of repetitions: 5                                                                                                                      Name: Facetime every day - Date: 7-1-2020, workday - Initial Time: 14:00 - Final Time: 15:00 - Frequency: Weekday; First Date: 3-1-2020, workday; number of repetitions: 5 - Frequency: Weekday; First Date: 3-1-2020, workday; number of repetitions: 5                   

In [37]:
-- 8.12
-- Finally, we call the actual main function using all the auxiliary depending on case
-- raise an error if date1 > date2 in the final function
-- apply listAgenda to Agenda in order to have it sorted before doing anything
-- for events with several preferences, we prioritize 1. persons 2. place 3. time

---------------------------------------------------
-- treating events with preferences
-- logic:

-- if there is a person preference
    -- if there are slots with the person in the preference we return the first one
    -- if there are no slots
        -- if there is a place preference we look for it
        -- if there is no place preferene and there is a time preference we look for it
        -- if there are no other preferences we schedule like a normal event
-- if there is no person preference and there is place preference
    -- if there are slots with that place we return the first one
    -- if there are no slots with that place
        -- if there is a time preference we look for a slot satisfying it
            -- if there is none, we schedule like an event without preference
        -- if there is no time preference, we schedule like an event without preference
-- if there is only time preference
    -- if there is a slot satisfying it, we schedule in that slot
    -- if not, we schedule like an event without preference

slotToPersons::Slot->[String]
slotToPersons (Slot _ _ _ (Persons p) _) = p
slotToPlace::Slot->String
slotToPlace (Slot _ _ _ _ p) = p

slotsHavingPerson::[Slot]->String->[Slot]
slotsHavingPerson [x] p
    | p `elem` slotToPersons x = [x]
    | otherwise = []
slotsHavingPerson (x:xs) p 
    | p `elem` slotToPersons x = x : slotsHavingPerson xs p
    | otherwise = slotsHavingPerson xs p

slotsHavingPlace::[Slot]->String->[Slot]
slotsHavingPlace [x] p
    | p == slotToPlace x = [x]
    | otherwise = []
slotsHavingPlace (x:xs) p 
    | p == slotToPlace x = x : slotsHavingPlace xs p
    | otherwise = slotsHavingPlace xs p

createEventPlace::Slot->Event->Agenda->Agenda
createEventPlace slot event (Agenda name events) = insert (event ++ [DPlace (getPlacePref event)] ++ [slotToInitial slot] ++ [slotToFinal slot]) (Agenda name events)

createEventPersons::Slot->Event->Agenda->Agenda
createEventPersons slot event (Agenda name events) = insert (event ++ [DPersons (Persons [getPersonsPref event])] ++ [slotToInitial slot] ++ [slotToFinal slot]) (Agenda name events)

scheduleEventPref'::Event->[Slot]->Agenda->Date->Date->Agenda
scheduleEventPref' event events (Agenda name e) date1 date2 
    | (getPersonsPref event /= "") && not (null(slotsHavingPerson events (getPersonsPref event))) = createEventPersons (head (slotsHavingPerson events (getPersonsPref event))) event (Agenda name e)
    | getPlacePref event == "" || null (slotsHavingPlace events (getPlacePref event)) = selectSlot4 event (Agenda name $ listAgenda $ Agenda name e) date1 date2
    | otherwise = createEventPlace (head (slotsHavingPlace events (getPlacePref event))) event (Agenda name e) 

---------------------------------------------------
-- treating events without non-time preferences
scheduleEventNotPref::Event->Agenda->Date->Date->Agenda
scheduleEventNotPref event (Agenda name events) date1 date2
    | isTherePreferences event = selectSlot4 event (Agenda name $ listAgenda $ Agenda name events) date1 date2
    | isThereInitial event = selectSlot1 event (Agenda name $ listAgenda $ Agenda name events) date1 date2
    | isThereDate event = selectSlot2 event (Agenda name $ listAgenda $ Agenda name events) date1 date2
    | otherwise = selectSlot3 event (Agenda name $ listAgenda $ Agenda name events) date1 date2 

---------------------------------------------------
-- bridge for events with non-time preferences
scheduleEventPref::Event->Agenda->Date->Date->Agenda
scheduleEventPref event (Agenda name events) date1 date2 = scheduleEventPref' event (availableThroughDates (getDuration event) (listAgenda (Agenda name events)) (getListDates date1 date2)) (Agenda name events) date1 date2

scheduleEvent'::Event->Agenda->Date->Date->Agenda
scheduleEvent' event (Agenda name events) date1 date2
    | date1 > date2 = error "The first input date must be prior to the second one" 
    | isTherePreferences event && (getPlacePref event /= "" || getPersonsPref event /= "") = scheduleEventPref event (Agenda name events) date1 date2
    | otherwise = scheduleEventNotPref event (Agenda name events) date1 date2
    
---------------------------------------------------
-- treating events with repetition
scheduleEventFreq::[Event]->Agenda->Date->Date->Agenda
scheduleEventFreq [x] agenda date1 date2 = scheduleEvent' x agenda date1 date2
scheduleEventFreq (x:xs) agenda date1 date2 = scheduleEventFreq xs (scheduleEvent' x agenda date1 date2) date1 date2

---------------------------------------------------
-- master function

scheduleEvent::Event->Agenda->Date->Date->Agenda
scheduleEvent event (Agenda name events) date1 date2
    | date1 > date2 = error "The first input date must be prior to the second one" 
    | isThereFrequency event = Agenda name (listAgenda (scheduleEventFreq (createFreqEvents event date1 date2) (Agenda name events) date1 date2))
    | otherwise = scheduleEvent' event (Agenda name events) date1 date2

In [38]:
-- 8.13
-- Testing

event1 = [DName "Exam", DDate (Date 3 1 2020 True), DInitial (Time 12 30), DFinal (Time 14 0), DPersons(Persons["Gonzalo","Jorge"]), DPlace "Leganes"]
event2 = [DName "Gym", DDate (Date 3 1 2020 True), DInitial (Time 19 0), DFinal (Time 21 0), DPersons(Persons["Cristobal"]), DPlace "Chamberi"]
event3 = [DName "Meeting", DDate (Date 3 1 2020 True), DInitial (Time 11 0), DFinal (Time 12 0), DPlace "Online", DPersons(Persons["Hugo"])]
event4 = [DName "Dinner", DDate (Date 3 1 2020 True), DInitial (Time 21 30), DFinal (Time 23 0), DPersons(Persons["Paula"]), DPlace"Casa"]
events = [event1,event2,event3,event4]
myAgenda = Agenda "Carlos" events
date1 = Date 2 1 2020 False
date2 = Date 4 1 2020 True

event5 = [DName "Running", DInitial(Time 12 00), DFinal (Time 18 00), DDate (Date 6 1 2020 True)]
events2 = events ++ [event5]
date3 = Date 31 1 2020 True
myAgenda2 = Agenda "Carlos" events2

---------------
-- error testing
-- printAgenda (scheduleEvent event myAgenda (Date 4 1 2020 True) (Date 3 1 2020 True))
-- works for invalid date input

---------------
-- non-preference testing

-- event = [DName "Reading", DDuration 60, DDate (Date 4 1 2020 True)]
-- works for no preference, partially defined in terms of date

-- event = [DName "Reading", DInitial (Time 17 15), DFinal (Time 18 30), DDate (Date 3 1 2020 True)]
-- works for no preference, fully defined that can be scheduled

-- event = [DName "Reading", DInitial (Time 17 15), DFinal (Time 19 15), DDate (Date 3 1 2020 True)]
-- works for no preference, fully defined that cannot be scheduled

-- event = [DName "Reading", DDuration 90]
-- works for no preference, undefined in terms of date


---------------
-- preference testing

-- event = [DName "Reading", DDuration 60, DPreferences (Preferences (Time 14 0) (Time 22 0) "Madrid" "Jorge"), DDate (Date 3 1 2020 True)]
-- works for person preference that can be satified

-- event = [DName "Reading", DDuration 30, DPreferences (Preferences (Time 10 0) (Time 23 30) "Casa" "Paco"), DDate (Date 3 1 2020 True)]
-- works for person preference that can't be satisfied, place preferene that can 

-- event = [DName "Reading", DDuration 60, DPreferences (Preferences (Time 08 30) (Time 09 30) "Madrid" "Paco"), DDate (Date 3 1 2020 True)]
-- works for person preference that can't be satisfied, place preferene that can't, time preference that can

-- event = [DName "Reading", DDuration 90, DPreferences (Preferences (Time 07 00) (Time 08 30) "Paco" ""), DDate (Date 3 1 2020 True)]
-- works for person preference that can't be satisfied, no place preference , time preference that can

-- event = [DName "Reading", DDuration 60, DPreferences (Preferences (Time 11 30) (Time 12 30) "" "Paco"), DDate (Date 3 1 2020 True)]
-- works for person preference that can't be satisfied, no place preference , time preference that can't

event = [DName "Reading", DDuration 45, DPreferences (Preferences (Time 09 0) (Time 23 30) "Chamberi" ""), DDate (Date 3 1 2020 True)]
-- works for place preference that can be satisfied, no person preference

-- event = [DName "Reading", DDuration 45, DPreferences (Preferences (Time 09 0) (Time 23 30) "Barcelona" ""), DDate (Date 3 1 2020 True)]
-- works for place preference that can't be satisfied, no person preference, time preference that can

-- event = [DName "Reading", DDuration 45, DPreferences (Preferences (Time 11 0) (Time 12 0) "Barcelona" ""), DDate (Date 3 1 2020 True)]
-- works for place preference that can't be satisfied, no person preference, time preference that can't

-- event = [DName "Reading", DDuration 60, DPreferences (Preferences (Time 18 0) (Time 19 0) "" ""), DDate (Date 3 1 2020 True)]
-- works for just time preference that can be satisfied

-- event = [DName "Reading", DDuration 90, DPreferences (Preferences (Time 18 0) (Time 19 30) "" ""), DDate (Date 3 1 2020 True)]
-- works for just time preference that can't be statisfied

---------------
-- frequency testing
event' = [DName "Facetime every weekday", DDate (Date 3 1 2020 True), DInitial (Time 14 0), DFinal (Time 15 0), DFrequency (NP Daily (Date 3 1 2020 True) 5)]
-- works for events that repeat themselves
-- createFreqEvents event' date1 date3

printAgenda (scheduleEvent event myAgenda date1 date2)
printAgenda (scheduleEvent event' myAgenda2 date1 date3)

"Carlos agenda info:                                                                                                             Name: Meeting - Date: 3-1-2020, workday - Initial Time: 11:00 - Final Time: 12:00 - Attendants: Hugo - Place: Online - Place: Online                                                                                                                      Name: Exam - Date: 3-1-2020, workday - Initial Time: 12:30 - Final Time: 14:00 - Attendants: Gonzalo, Jorge - Place: Leganes - Place: Leganes                                                                                                                      Name: Reading - Date: 3-1-2020, workday - Initial Time: 18:15 - Final Time: 19:00 - Duration: 45 minutes - Place: Chamberi - Place: Chamberi - Preferences: Earliest Time: 09:00, Latest Time: 23:30, Place: Chamberi, Person:                                                                                                                       Name: Gym - Date: 3-1

"Carlos agenda info:                                                                                                             Name: Meeting - Date: 3-1-2020, workday - Initial Time: 11:00 - Final Time: 12:00 - Attendants: Hugo - Place: Online - Place: Online                                                                                                                      Name: Exam - Date: 3-1-2020, workday - Initial Time: 12:30 - Final Time: 14:00 - Attendants: Gonzalo, Jorge - Place: Leganes - Place: Leganes                                                                                                                      Name: Facetime every weekday - Date: 3-1-2020, workday - Initial Time: 14:00 - Final Time: 15:00 - Frequency: Daily; First Date: 3-1-2020, workday; number of repetitions: 5 - Frequency: Daily; First Date: 3-1-2020, workday; number of repetitions: 5                                                                                                                  

### Exercise 9


In this section we create a function allPossibleSchedules Event Agenda StartDate EndDate that schedules all the events in an input agendas with all possible schedules for the event that respect preferences.

We reuse and slightly tweak methods from the previous section, again depending on the decision tree of scheduling events based on present preferences.

Based on the statement we must return all the possible schedules for the event that respect preferences, this leads us to developed a more constrained approach than that of the previous exercises where we dropped preferences iteratively until we found an empty slot for the event, even if it does not satisfy some or all input preferences.

In this case we look for a slot that satisfies all preferences and, if there is none, the list of agendas will be empty, since there is no possible agenda to choose from. 

For events fully defined in terms of date, we either return one or zero agendas, depending on whether the time for the event is already occupied or not, if the events are partially defined, i.e, date and duration are present, we return as many agendas as slots with that duration on that date that are right before or after an already scheduled event.

If the event is not defined in terms of date, we do the same as above but with every single date between the input dates of the function.

Also if the start input date is greater than the end input date we return an error message.

In [39]:
-- 9 

-- APPROACH: reuse and slightly tweak methods and functions from 8 to,
-- again depending on a decision tree depending on event preferences and degree of time definition,
-- get a list of different possible agendas. 

-- First we get the potential slots, with special attention for events with preferences
-- since wehave to make sure those slots satisfy such preferences (***), then for each potential slot
-- we add to the return list an agenda where the event is placed in that slot

-- *** from the exercise statement we are told "all the possible schedules for the event that respect preferences"
-- This forces us to apply a different, more strict approach than in exercise 8, where preferences could be
-- iteratively ignored such that we at least found an empty slot for the event, even if it does not satisfy
-- the main preference(s) or even none of them
-- Here on the other hand, we must look for a slot that satisfies all preferences and, if there is none,
-- the list of agendas will be empty since there is no possible agenda to choose from, since no agenda would
-- respect the event preferences

-- for events fully defined in terms of date, we either return one or zero agendas depending on whether the
-- time for the event is already occupied or not

possibleAgendas2::Event->Agenda->Date->Date->[Agenda]
possibleAgendas2 event (Agenda name events) date1 date2
    | anyIncluded (timesToSlot event) (getSlots events) = []
    | otherwise = [insert event (Agenda name events)]
    
-- for events partially defined (date+duration), we return as many agendas as slots with that duration 
-- on that date that are right before or after an already scheduled event (otherwise we would have potentially hundreds)

-- for events not defined in terms of date, we do the same as above but with every single date between
-- the input dates of the main function
    
readyToInsert2::Event->Slot->Event
readyToInsert2 event slot 
    | isThereDate event = event ++ [slotToInitial slot] ++ [slotToFinal slot]
    | otherwise = event ++ [slotToDate slot] ++ [slotToInitial slot] ++ [slotToFinal slot]

possibleAgendas3::Event->[Slot]->Agenda->[Agenda]
possibleAgendas3 event [x] (Agenda name events) = [insert (readyToInsert2 event x) (Agenda name events)]
possibleAgendas3 event (x:xs) (Agenda name events) = insert (readyToInsert2 event x) (Agenda name events) : possibleAgendas3 event xs (Agenda name events)

-- for events with preferences, we get the preferences from the event. If some of them is empty (recall "" implies
-- no preference), we will not take it into account when looking for a slot satisfyinhg those preferences: just like
-- in the previous exercise, we get the preferences that events can satisfy by the place and Persons of events
-- scheduled right before or after the slot

-- we don't know if an event has all preferences specified so, in case of scheduling it, we need to make sure
-- what descriptors to add to the event based on what preferences it has

readyToInsert::Event->Slot->Event
readyToInsert event slot 
    | isThereDate event && getPlacePref event == "" && getPersonsPref event == "" = event ++ [slotToInitial slot] ++ [slotToFinal slot]
    | isThereDate event && getPlacePref event /= "" && getPersonsPref event == "" = event ++ [slotToInitial slot] ++ [slotToFinal slot] ++[DPlace (getPlacePref event)]
    | isThereDate event && getPlacePref event == "" && getPersonsPref event /= "" = event ++ [slotToInitial slot] ++ [slotToFinal slot] ++[DPersons (Persons [getPersonsPref event])]
    | isThereDate event && getPlacePref event /= "" && getPersonsPref event /= "" = event ++ [slotToInitial slot] ++ [slotToFinal slot] ++[DPersons (Persons [getPersonsPref event])] ++ [DPlace (getPlacePref event)]
    | not (isThereDate event) && getPlacePref event == "" && getPersonsPref event == "" = event ++ [slotToDate slot] ++ [slotToInitial slot] ++ [slotToFinal slot]
    | not (isThereDate event) && getPlacePref event /= "" && getPersonsPref event == "" = event ++ [slotToDate slot] ++ [slotToInitial slot] ++ [slotToFinal slot] ++[DPlace (getPlacePref event)]
    | not (isThereDate event) && getPlacePref event == "" && getPersonsPref event /= "" = event ++ [slotToDate slot] ++ [slotToInitial slot] ++ [slotToFinal slot] ++[DPersons (Persons [getPersonsPref event])]
    | otherwise = event ++ [slotToDate slot] ++ [slotToInitial slot] ++ [slotToFinal slot] ++[DPersons (Persons [getPersonsPref event])] ++[DPlace (getPlacePref event)] 
    
includesAllPref::Slot->Event->Bool
includesAllPref slot event
    | (not (isThereDate event) || (isThereDate event && getDate event == getDate[slotToDate slot])) && getInitial[slotToInitial slot] >= initialPref (getPreferences event) && getFinal[slotToFinal slot] <= finalPref(getPreferences event) && (getPersonsPref event == "" || (getPersonsPref event `elem` slotToPersons slot)) && (getPlacePref event == "" || (getPlacePref event == slotToPlace slot)) = True
    | otherwise = False
    
possibleAgendas1::Event->[Slot]->Agenda->[Agenda]
possibleAgendas1 event [x] (Agenda name events)
    | includesAllPref x event = [insert (readyToInsert event x) (Agenda name events)]
    | otherwise = []
possibleAgendas1 event (x:xs) (Agenda name events)
    | includesAllPref x event = insert (readyToInsert event x) (Agenda name events) : possibleAgendas1 event xs (Agenda name events)
    | otherwise = possibleAgendas1 event xs (Agenda name events)
    
-- master function
allPossibleSchedules::Event->Agenda->Date->Date->[Agenda]
allPossibleSchedules event (Agenda name events) date1 date2
    -- error
    | date1 > date2 = error "The first input date must be prior to the second one" 
    -- preferences
    | isTherePreferences event && not (isThereDate event) = possibleAgendas1 event (availableThroughDates (getDuration event) events (getListDates date1 date2)) (Agenda name events)
    | isTherePreferences event && isThereDate event = possibleAgendas1 event (availableThroughDates (getDuration event) events (getListDates (getDate event) (getDate event))) (Agenda name events)
    -- no preferences, fully defined
    | isThereInitial event = possibleAgendas2 event (Agenda name events) date1 date2
    -- no pref, partially defined
    | isThereDate event = possibleAgendas3 event (availableThroughDates (getDuration event) events (getListDates (getDate event) (getDate event))) (Agenda name events)
    -- no pref, undefined in terms of time
    | otherwise = possibleAgendas3 event (availableThroughDates (getDuration event) events (getListDates date1 date2)) (Agenda name events)


In [40]:
-- 9.testing
printAgendas::[Agenda]->String
printAgendas [x] = printAgenda x
printAgendas (x:xs) = printAgenda x ++ ".........................................................................................................................                                                                                                          " ++ printAgendas xs
printAgendas [] = "No possible agendas, no available slot can fit the event"
 
event1 = [DName "Exam", DDate (Date 3 1 2020 True), DInitial (Time 12 30), DFinal (Time 14 0), DPersons(Persons["Gonzalo","Jorge"]), DPlace "Leganes"]
event2 = [DName "Gym", DDate (Date 3 1 2020 True), DInitial (Time 19 0), DFinal (Time 21 0), DPersons(Persons["Cristobal"]), DPlace "Chamberi"]
event3 = [DName "Meeting", DDate (Date 3 1 2020 True), DInitial (Time 11 0), DFinal (Time 12 0), DPlace "Online", DPersons(Persons["Hugo"])]
event4 = [DName "Dinner", DDate (Date 3 1 2020 True), DInitial (Time 21 30), DFinal (Time 23 0), DPersons(Persons["Paula"]), DPlace"Casa"]
events = [event1,event2,event3,event4]
myAgenda = Agenda "Carlos" events
date1 = Date 3 1 2020 True
date2 = Date 4 1 2020 True

---------------
-- error testing
-- printAgendas (allPossibleSchedules event myAgenda date2 date1)
-- works for invalid date input

---------------
-- non-preference testing

-- event = [DName "Reading", DDuration 60, DDate (Date 4 1 2020 True)]
-- works for no preference, partially defined in terms of date

-- event = [DName "Reading", DInitial (Time 17 15), DFinal (Time 18 30), DDate (Date 3 1 2020 True)]
-- works for no preference, fully defined that can be scheduled

-- event = [DName "Reading", DInitial (Time 17 15), DFinal (Time 19 15), DDate (Date 3 1 2020 True)]
-- works for no preference, fully defined that cannot be scheduled

-- event = [DName "Reading", DDuration 90]
-- works for no preference, undefined in terms of date


---------------
-- preference testing

event = [DName "Reading", DDuration 25, DPreferences (Preferences (Time 8 0) (Time 24 0) "Leganes" "Jorge"), DDate (Date 3 1 2020 True)]
-- event = [DName "Reading", DDuration 70, DPreferences (Preferences (Time 08 0) (Time 24 0) "Leganes" "")]
-- event = [DName "Reading", DDuration 120, DPreferences (Preferences (Time 19 0) (Time 24 0) "" "")]
-- works for preferences that can be satified

-- event = [DName "Reading", DDuration 60, DPreferences (Preferences (Time 8 0) (Time 14 0) "Leganes" "Jorge"), DDate (Date 3 1 2020 True)]
-- event = [DName "Reading", DDuration 70, DPreferences (Preferences (Time 9 0) (Time 7 0) "" "")]
-- event = [DName "Reading", DDuration 120, DPreferences (Preferences (Time 19 0) (Time 24 0) "" ""), DDate (Date 3 1 2020 True)]
-- event = [DName "Reading", DDuration 60, DPreferences (Preferences (Time 0 0) (Time 24 0) "Chamberi" "Jorge"), DDate (Date 3 1 2020 True)]
-- works for preferences that can't be satified

printAgendas (allPossibleSchedules event myAgenda date1 date2)

"Carlos agenda info:                                                                                                             Name: Meeting - Date: 3-1-2020, workday - Initial Time: 11:00 - Final Time: 12:00 - Attendants: Hugo - Place: Online - Place: Online                                                                                                                      Name: Reading - Date: 3-1-2020, workday - Initial Time: 12:05 - Final Time: 12:30 - Duration: 25 minutes - Attendants: Jorge - Place: Leganes - Place: Leganes - Preferences: Earliest Time: 08:00, Latest Time: 24:00, Place: Leganes, Person: Jorge                                                                                                                      Name: Exam - Date: 3-1-2020, workday - Initial Time: 12:30 - Final Time: 14:00 - Attendants: Gonzalo, Jorge - Place: Leganes - Place: Leganes                                                                                                                     

### Exercise 10

In this exercise we want to schedule a list of input events in a list of input agendas so that they can take place simultaneously, within some date bounds, returning a list with the new agendas.

Our first observation is that in this exercise in very few cases will we not be able to schedule an event, since it would have to be very long, on a fixed date an that date be very occupied in the owner's agendas.

The idea is to first call on an event and agenda list basis, and then iterate through the list of events, getting a list of available slots for the event on each agenda, point at which we reuse methods from section 9, we get the intersection of those agendas, and if it is not empty, we update each agenda with the scheduled event on the first position of the slot intersection, and if it is in fact empty, we return the list of agendas as is.

In [41]:
-- 10. scheduleSharedEvents [Event] [Agenda] StartDate EndDate

-- Note that for this exercise, in theory, it would be very hard to NOT be able to schedule an event, 
-- unless it is very long + fixed on a date + on that date both owner of agendas have a lot of events scheduled.

-- What that means is, if there are 2 hours that both of them have free and the event is 30 minutes, 
-- there are up to 90 (2*60-90) times to schedule it that work
-- It does not make sense to work like this because:
-- 1. No one is scheduling tasks at times like “18:23-19:13”
-- 2. Scheduling it right in the middle of two events FOR ALL OWNERS is bad because it creates holes in the agenda 
-- that are hard to fill, which can be useful time if instead we focus on scheduling events 
-- right before or right after what is already scheduled

-- Therefore, it is fine for the rest of the owners to adapt to a time slot that makes sense for one of the owners 
-- according to the logic we just described, but we are not going to go through the (anywhere up to 1440) 
-- possible time frames in a day just to find one that complicates everyones’ agenda.

-- approach
-- First we call on an Event [Agenda] basis and then we will iterate the method created for that level over the
-- list of Events. 
-- We get a list of available slots for the event on each agenda, where we can reimplement methods from part 9
-- we get the intersection of those agendas. If it not empty, we update each agenda with the event on that
-- the first position of the intersection of slots. If it is, we keep the agendas the same.

-- 1. for each agenda in the input, get the allPossibleSchedules of the event in the agenda
-- 2. for each agenda in allPossibleAgendas:
    -- use searchEvent to find the event already inserted 
    -- use getSlot to create a slot from the event data
-- 3. create a list with the slots of the event for each agenda

getEventFromAgenda10::Agenda->Event
getEventFromAgenda10 (Agenda name events) 
    | null events = []
    | otherwise = head events
    
possibleSlot::Agenda->Event->[Slot]
possibleSlot agenda event
    | null event = []
    | otherwise = getSlots [getEventFromAgenda10 (searchEvents event agenda)]
    
allPossibleSlots::[Agenda]->Event->[Slot]
allPossibleSlots [] event = []
allPossibleSlots [x] event = possibleSlot x event
allPossibleSlots (x:xs) event = possibleSlot x event ++ allPossibleSlots xs event

-- call that function as 
-- allPossibleSlots (allPossibleSchedules event agenda date1 date2) event
-- for each agenda to get their list of slots

allPossibleSlotLists::[Agenda]->Event->Date->Date->[[Slot]]
allPossibleSlotLists xs event date1 date2 = [allPossibleSlots (allPossibleSchedules event a date1 date2) event | a <- xs]

-- 4. get the intersection of lists of slots
eqSlot::Slot->Slot->Bool
eqSlot (Slot i1 f1 d1 pl1 pe1) (Slot i2 f2 d2 pl2 pe2) = i1==i2 && f1==f2 && d1==d2
instance Eq Slot where
    slot1==slot2 = eqSlot slot1 slot2
    
inAll::Slot->[[Slot]]->Bool
inAll slot (x:xs)
    | null x = False
    | slot `elem` x = inAll slot xs
    | otherwise = False
inAll slot [x]
    | null x = False
    | slot `elem` x = True
    | otherwise = False
inAll slot [] = True
    
intersection::[Slot]->[[Slot]]->[Slot]
intersection xs ys  = [ a | a<-xs, inAll a ys ]

-- 5. pick the first slot and update agendas if list not empty, agendas stays the same if list empty

readyToInsert'::Event->Slot->Event
readyToInsert' event slot
    | slotToPersons slot /= [] || slotToPlace slot /= "" = readyToInsert event slot
    | otherwise = readyToInsert2 event slot
    
getCommonEvent::[Agenda]->Event->Date->Date->Event
getCommonEvent xs event date1 date2
    | null (intersection (head(allPossibleSlotLists xs event date1 date2)) (tail(allPossibleSlotLists xs event date1 date2))) = []
    | otherwise = readyToInsert' event (head (intersection (head(allPossibleSlotLists xs event date1 date2)) (tail(allPossibleSlotLists xs event date1 date2))))
    
-- 6. pass updated agendas to the next event
insertInOne::Agenda->Event->Agenda
insertInOne agenda event
    | null event = agenda
    | otherwise = insert event agenda
    
insertInAll::[Agenda]->Event->Date->Date->[Agenda]
insertInAll xs event date1 date2 = [insertInOne a (getCommonEvent xs event date1 date2) | a <- xs ]

-- We pass the list of updated agendas to next event and iterate
scheduleSharedEvents::[Event]->[Agenda]->Date->Date->[Agenda]
scheduleSharedEvents [x] ys date1 date2
    | date2 < date1 = error "The first input date must be prior to the second one" 
    | otherwise = insertInAll ys x date1 date2
scheduleSharedEvents (x:xs) ys date1 date2
    | date2 < date1 = error "The first input date must be prior to the second one" 
    | otherwise = scheduleSharedEvents xs (insertInAll ys x date1 date2) date1 date2

In [42]:
-- 10.testing
 
event1 = [DName "Exam", DDate (Date 3 1 2020 True), DInitial (Time 12 30), DFinal (Time 14 0), DPersons(Persons["Carlos","Jorge","Faustino"]), DPlace "Leganes"]
event2 = [DName "Gym", DDate (Date 3 1 2020 True), DInitial (Time 19 0), DFinal (Time 21 0), DPersons(Persons["Carlos","Jorge","Mengano", "Faustino"]), DPlace "Chamberi"]
event3 = [DName "Meeting", DDate (Date 3 1 2020 True), DInitial (Time 11 0), DFinal (Time 12 0), DPersons(Persons["Carlos","Mengano","Faustino"]),  DPlace "Online"]
event4 = [DName "Dinner", DDate (Date 3 1 2020 True), DInitial (Time 21 30), DFinal (Time 23 0), DPersons(Persons["Jorge","Mengano","Faustino"]), DPlace"Bel Mondo"]

-- for making sure the logic works, but doesn't leave room for much
event5 = [DName "Filling Mengano's Agenda such that nothing can be scheduled that day", DDate (Date 3 1 2020 True), DInitial (Time 2 30), DFinal (Time 24 0), DPersons(Persons["Mengano"])]

agenda1 = Agenda "Carlos" [event1,event2,event3]
agenda2 = Agenda "Jorge" [event1,event2,event4]
agenda3 = Agenda "Mengano" [event2,event3,event4]
-- agenda3 = Agenda "Mengano" [event5]
agenda4 = Agenda "Faustino" [event1, event2, event3, event4]

agendas = [agenda1, agenda2, agenda3, agenda4]

newevent1 = [DName "Read Harry Potter", DDuration 60, DDate (Date 3 1 2020 True)]
newevent2 = [DName "Read Hunger Games", DInitial (Time 17 15), DFinal (Time 18 30), DDate (Date 3 1 2020 True)]
newevent3 = [DName "Read 48 Laws of Power", DInitial (Time 17 15), DFinal (Time 19 15), DDate (Date 3 1 2020 True)]
newevent4 = [DName "Read Cracking the Coding Interview", DDuration 90, DPreferences (Preferences (Time 8 0) (Time 24 0) "" "")]
newevent5 = [DName "Read 11 Rules For Life", DDuration 25, DPreferences (Preferences (Time 8 0) (Time 24 0) "Leganes" "Jorge"), DDate (Date 3 1 2020 True)]
newevent6 = [DName "Read Meditations", DDuration 70, DPreferences (Preferences (Time 08 0) (Time 24 0) "Leganes" "")]
newevent7 = [DName "Do nothing", DDuration 120, DPreferences (Preferences (Time 19 0) (Time 24 0) "" "")]
newevent8 = [DName "Read whatever they want", DDuration 60, DPreferences (Preferences (Time 8 0) (Time 14 0) "Leganes" "Jorge"), DDate (Date 3 1 2020 True)]
newevent9 = [DName "Read Haskell questions on Stack Overflow", DDuration 70, DPreferences (Preferences (Time 9 0) (Time 7 0) "" "")]
newevent10 = [DName "Read Rich Dad, Poor Dad", DDuration 120, DPreferences (Preferences (Time 19 0) (Time 24 0) "" ""), DDate (Date 3 1 2020 True)]

date1 = Date 3 1 2020 True
date2 = Date 3 1 2020 True

---------------
-- error testing
-- printAgendas (scheduleSharedEvents newevents agendas date2 date1)
-- works for invalid date input

-- nice example showing the dynamic:
-- do newevents = [newevent6] to be able to focus on one event at a time. Start with the current test data
-- run it, then change the gym place to Leganes and run it again. Observ how it changes, our approach worked.

-- nice example, even more complex and dynamic, showing the full power of the exercise
-- 4 friends want to read as many books in a day as possible
-- they estimate for how much time they want to read each and they may add some additional prefferences
-- the order of readings in th list [events] is the preference order for the books, since it is the order that will be followed
-- for filling the schedule

newevents = [newevent1,newevent2,newevent3,newevent4,newevent5,newevent6,newevent7,newevent8,newevent9,newevent10]
printAgendas (scheduleSharedEvents newevents agendas date1 date2)

-- it turns out that on that particular date, given their agendas, schedules, characteristics of events and preferences,
-- the 4 friends can get together to read Harry Potter, Hunger Games and Cracking the Coding Interview

"Carlos agenda info:                                                                                                             Name: Read Harry Potter - Date: 3-1-2020, workday - Initial Time: 00:00 - Final Time: 01:00 - Duration: 60 minutes                                                                                                                      Name: Meeting - Date: 3-1-2020, workday - Initial Time: 11:00 - Final Time: 12:00 - Attendants: Carlos, Mengano, Faustino - Place: Online - Place: Online                                                                                                                      Name: Exam - Date: 3-1-2020, workday - Initial Time: 12:30 - Final Time: 14:00 - Attendants: Carlos, Jorge, Faustino - Place: Leganes - Place: Leganes                                                                                                                      Name: Read Cracking the Coding Interview - Date: 3-1-2020, workday - Initial Time: 15:45 - Final Tim

### Exercise 11

In this section we develop functions that allow the user to create and see an Agenda, and to search, create, insert, delete and schedule Events in one or several Agendas.

The first thing we do is define functions to read the different types that the descriptors can take (name, place, duration, initial, final, persons ...), while also doing error checking in particular checking that a possible input date is valid, valid times and categories. Then we defined a recursive function to read descriptors until the user desires to stop, this allows the user to create an event in an IO fashion. Once we have this $readDescriptors$ function we define a $createEvent$ function to perform checking on the input descriptors, possible input duration must match the difference between initial and final time if the three are specified, if we have two names or two places we will by default remove the second one from the list.

Once we have this, we employ recursion once more to create an agenda with a name and a series of calls to the createEvent function. Before creating the agenda we pass the read event list through a checker that ensures that the owner is part of the persons descriptor for each event and is at the first place.

Once the agenda and event creation have been taken care of, we focus on visualization, in order to see the contents of the agenda we define three IO based functions, the first one, $printEvent$ to see descriptors in an event, which just receives an event and prints descriptor by descriptor, the second one $printEvents$ which receives a list of events and for each item calls the first function, and the third printAgenda which just prints the name and then makes a call to the second function passing its event list as input.

Now in order to insert, delete and schedule an event exploiting IO we will work on top of the methods defined in sections 5, 7 and 8 respectively. In order to insert an event into an agenda we remove the event as parameter and read it from the user, once we have it read, we just call the method in section 5. This same idea is followed for deletion and insertion as well.

In [43]:
-- 11. Creation of functions to allow the user to see, create and insert in Agendas
-- idea:
-- first import control monad, necessary to get the functor and monad classes
-- then function to read descriptors, create and append one to descriptor list depending on the type
-- for each of the descriptors types we define functions to read them
-- need error checking dates, once it is read
-- check person order to be correct change otherwise

 --11.1 defining reading functions for every type of descriptor
import Control.Monad

--auxiliary function for converting between string and integer
toInt::String->Int
toInt s = read s ::Int


-- defining operator to check if input is within some bounds
(=?)::Int->(Int, Int)->Bool
(=?) x (a, b) = x >= a && x <= b

--to read an integer
readInt::IO Int
readInt = do
    toInt <$> getLine
    

-- reading category checking that input value is within admissible domain
readCategory::IO String
readCategory = do
    cat <- getLine
    if cat `notElem` ["personal", "health", "work", "management"]
        then error "invalid category type"
        else return  cat



-- function to check if the input date is valid
checkDate::Date->Bool
checkDate (Date d m y _) = d =? (1, daysInMonth m y) && m =? (1,12) && y > 0


-- defining auxiliary function to check date
daysInMonth :: Int -> Int -> Int
daysInMonth m y
  | m == 2 = if isleap y then 29 else 28
  | m `elem` [4, 6, 9, 11] = 30
  | otherwise = 31
  

-- reading date
readDate:: IO Date
readDate = do
    putStrLn "Day: "
    day <- getLine
    putStrLn "Month: "
    month <- getLine
    putStrLn "Year: "
    year <- getLine
    putStrLn "workday: "
    wk <- getLine
    if wk == "True"
        then return (Date (toInt day) (toInt month) (toInt year) True)
        else return (Date (toInt day) (toInt month) (toInt year) False)


-- function to read times
readTime::IO Time
readTime = do
    putStrLn "Read hours: "
    h <- readInt
    putStrLn "Read minutes: "
    m <- readInt
    if not ( h =? (0, 23)) && (m =? (0, 59))
        then error "invalid times"
        else return (Time h m)

-- -- function to read Persons
readPersons::IO[String]
readPersons = do
    person <- getLine
    if person == "stop"
        then return []
        else do
            persons <- readPersons
            return (person:persons)


-- -- function to read place
readPlace::IO String
readPlace = getLine

-- auxiliary function to check period and create frequency object accordingly
case_checkf::String->Date->Int->Frequency
case_checkf f Date{} n
    | f == "daily" = NP Daily Date {} n
    | f == "weekly" = NP Weekly Date {} n
    | f == "weekday" = NP Weekday Date {} n
    | otherwise = error "invalid frequency of NP event"

-- read recurrence time and check invalid, from frequency
readRec::IO String
readRec = do
    rec <- getLine
    if rec `notElem` ["weekly", "daily", "weekday"]
        then error "invalid category type"
        else return  rec


readFreq::IO Frequency
readFreq = do
    freq <- getLine
    if freq == "punctual"
        then return Punctual
        else do
            putStrLn "Introduce recurrency time: "
            f <- readRec
            putStrLn "Introduce first date: "
            d <- readDate
            putStrLn "Introduce repetitions: "
            case_checkf f d <$> readInt
            --turn case_checkf f d n


readPreferences:: IO Preferences
readPreferences = do 
    putStrLn "Introduce earliest time: "
    e <- readTime
    putStrLn "Introduce latest time: "
    l <- readTime
    putStrLn "Introduce location preference: "
    p <- readPlace
    putStrLn "Introduce person preference: "
    Preferences e l p <$> getLine

In [44]:
-- 11.2 function to read a descriptor (ask user for descriptor type and read depending on descriptor type)
-- checks category is valid and date


rdesc::String->IO Descriptor
rdesc s 
    | s == "name" = DName <$> getLine
    | s == "category" = do
        cat <- readCategory
        if cat == "personal" then return (DCategory Personal)
        else if cat == "health" then return (DCategory Health)
        else if cat == "work" then return (DCategory Work)
        else if cat == "management" then return (DCategory Management)
        else error "invalid category"
    | s == "date" = do
        date <- readDate
        if not ( checkDate date) then error "invalid date"
        else return (DDate date)
    | s == "initial" = DInitial <$> readTime
    | s == "final" = DFinal <$> readTime
    | s == "duration" = DDuration <$> readInt
    | s == "persons" = DPersons . Persons <$> readPersons
    | s == "place" = DPlace <$> readPlace
    | s == "frequency" = DFrequency <$> readFreq
    | s == "preferences" = DPreferences <$> readPreferences
    | otherwise = error "invalid descriptor type"
    
    

In [45]:
--11.2.2 now we define a function to readDescriptors, recursively

readDescriptors::IO[Descriptor]
readDescriptors = do
    input <- getLine
    if input == "stop" 
        then return []
        else do
            desc <- rdesc input
            descs <- readDescriptors
            return $ desc:descs

In [47]:
-- reading a list of descriptors
readDescriptors

Day: 
Month: 
Year: 
workday: 
[Name: poker,Attendants: elon musk, jorge, carlos,Date: 26-12-2021, workday,Place: casino]

In [48]:
-- 11.3 function to create a list of events, and check for errors in event
validduration::Event->Bool
validduration e
    | fullyscheduled e && (diff (getFinal e) (getInitial e) /= getDuration e) = False
    | otherwise = True

fullyscheduled::Event->Bool
fullyscheduled ev = isThereInitial ev && isThereFinal ev && isThereDuration ev


-- base case just one event
createEvent::IO Event
createEvent = do
    print "Reading descriptors"
    ev <- readDescriptors
    if not (validduration ev)
        then error "invalid duration"
        else return ev
    
--read events
readEvents::IO[Event]
readEvents = do
    putStrLn "Create event? (y|n): "
    r <- getLine
    if r == "n"
        then return []
        else do
            putStrLn "reading event"
            ev <- createEvent
            events <- readEvents
            return $ ev : events
    

In [48]:
-- reading a series of events
-- readEvents
readEvents

Create event? (y|n): 
[]

In [49]:
--11.4 function to create an agenda with as many elements as the user desires
-- find index person in list
findIndex::String->[String]->Int
findIndex n l = head [s | s <- [0..length l], l!!s == n]

-- remove occurence from person list
rmAt::Int->[String]->[String]
rmAt n l = take n l ++ drop (n + 1) l

--find position of person descriptors
findPersons::[Descriptor]->Int
findPersons l = head [s |s <- [0..length l], isPersons(l!!s)]

-- remove at index on descriptor list
rmAtD::Int->[Descriptor]->[Descriptor]
rmAtD n l = take n l ++ drop (n + 1) l


-- check whether the event needs modifying and return new person descriptor
modif::String->[String]->[String]
modif name per
    | name `elem` per = name : rmAt (findIndex name per) per
    | otherwise = name:per
    
    
-- works at an event level
checkPersonInEvents::String->[Event]->[Event]
checkPersonInEvents name [] = []
checkPersonInEvents name [x] 
    | isTherePersons x && name /= head(getPersons x)= [DPersons(Persons (modif name (getPersons x))):rmAtD (findPersons x) x]
    | otherwise = [x] -- no persons in event, we return unchanged
checkPersonInEvents name (x:xs)
    | isTherePersons x = (DPersons(Persons (modif name (getPersons x))):rmAtD (findPersons x) x) : checkPersonInEvents name xs
    | otherwise = checkPersonInEvents name xs -- no persons in event move to next one
    



In [50]:
-- to intertwine a call to the checking functions
readAgendaEvs::String->IO [Event]
readAgendaEvs n = checkPersonInEvents n <$> readEvents

In [51]:
createAgenda:: IO Agenda
createAgenda= do
    putStrLn "set owner name: "
    name <- getLine
    Agenda name <$> readAgendaEvs name

In [62]:
-- creating an agenda
a <- createAgenda
printAgenda a

set owner name: 
Create event? (y|n): 
reading event
"Reading descriptors"
Read hours: 
Read minutes: 
Read hours: 
Read minutes: 
Create event? (y|n):

"Jorge agenda info:                                                                                                             Name: chilling - Initial Time: 18:00 - Final Time: 19:00"

In [63]:
--11.5 At this point agenda creation is done, we now focus on visualization

printEvent::Event->IO()
printEvent [] = print "\n"
printEvent [x] = print x
printEvent (x:xs) = do
    print x
    printEvent xs
    
printEvents::[Event]->IO()
printEvents [] = print ""
printEvents [x] = printEvent x
printEvents (x:xs) = do
    printEvent x
    putStrLn "\n"
    printEvents xs


printAgendaio::Agenda->IO()
printAgendaio a@(Agenda n ev) = do
    print n
    printEvents ev


In [64]:
-- testing io display of agenda
printAgendaio agenda
-- printAgendaio a

"Carlos"
Date: 22-10-2020, workday
Initial Time: 15:00
Final Time: 16:00


Date: 22-10-2020, workday
Initial Time: 18:00
Final Time: 19:00

In [54]:
scheduleEventio::Agenda->Date->Date->IO Agenda
scheduleEventio (Agenda name events) date1 date2 = do
    event <- createEvent
    return $ scheduleEvent event (Agenda name events) date1 date2

In [60]:
-- testing function scheduling
ag <- scheduleEventio agenda (Date 1 1 2020 True) (Date 12 12 2022 True)
printAgendaio ag

"Reading descriptors"
Read hours: 
Read minutes: 
Read hours: 
Read minutes:

"Carlos"
Date: 22-10-2020, workday
Initial Time: 15:00
Final Time: 16:00


Date: 22-10-2020, workday
Initial Time: 18:00
Final Time: 19:00


Initial Time: 17:00
Final Time: 19:30
Place: gym

In [65]:
insertEventio::Agenda->IO Agenda
insertEventio a@(Agenda name events) = do
    event <- createEvent
    return $ insert event a

In [66]:
ag <- insertEventio a
printAgendaio ag

"Reading descriptors"
Read hours: 
Read minutes: 
Read hours: 
Read minutes:

"Jorge"
Name: chilling
Initial Time: 18:00
Final Time: 19:00


Name: dinner
Initial Time: 21:00
Final Time: 23:00

In [67]:
deleteEventio::Agenda->IO Agenda
deleteEventio (Agenda name events) = do
    event <- createEvent
    return $ deleteEvents event (Agenda name events)

In [68]:
agd <- deleteEventio ag
printAgendaio agd

"Reading descriptors"
Read hours: 
Read minutes: 
Read hours: 
Read minutes:

"Jorge"
Name: chilling
Initial Time: 18:00
Final Time: 19:00

## DIFFICULTIES ENCOUNTERED

The most difficult task was by far exercise 8. Generating the slots and then removing the occupied ones was particularly hard as we had events that did not finish exactly at the end of a slot, and therefore, in order to improve our scheduling, we had to recompute slots to not lose valuable time in the date.

Also in this exercise, matching the slots to the input preferences was quite a cumbersome task as we had to resort to checking one by one the different preferences against the slots was rather difficult as we had to design a decision tree to efficiently do so.

Another difficulty we encountered is in exercise 9, as at last we needed to define a time '24:00' because of the overlap that using '0:00' caused between the possible end of an event and that start of the next day.

Also a hurdle we encountered is the recurrent need of a great deal of data extraction methods, and data reading methods in exercise 11, in order to be able solve the tasks which for us was particularly tiring as being used to imperative programming languages such as Python, for this same reason we struggled, particularly at first to work in the scale set by the statement without being able to to loop, having to  resort to recursion a great many times throughout the completion of the project.

In the case of exercises 8 and 9, for every descriptor X, we were forced to develop a large amount of functions checking if X exists in an event, if a descriptor is type X, returning x, going from data inside a Slot to a data type X and vice-versa, etc. That definitely made the process slower

In general, specially from exercise 8 onwards, debugging was really hard given that:

(1) We had to develop a structure of interconected functions, such that the inputs of some functions are outputs of others instead of having just one or a few functions where inside can freely create and manipulate any data type

(2) Specifying the types for every function such that everything matches cohesively along the passing of inputs and outputs was a big challenge, since we had to do it in abstraction mode, given that we could not test with practical examples until the program compiled succesfully

(3) The <interactive> error messages forced us to go back, reconsider and change other blocks of the code, since they made us find on later parts of the project that our previous methods were not efficient enough
    
(4) The non-exhaustive definition of some functions really made us pay attention to every possible scenario of a function's use, even those out of what we are trying to achieve

Testing was also harder than we are used to, as once compilation is succesfull does not necessarily mean the result will be correct. We encountered that many times and had to test in depth calling lower and lower level functions to find out where the error was.

Overall, the quite tedious coding process along with the long and exhaustive debugging and testing aspects of it and especiallly the new paradigm of thinking and new levels of abstraction we had to reach, made this one of the hardest projects we have faced during our completion of the degree. Nevertheless, we are extremely proud that we didn't get complacent and didn't stop without covering the hardest functionalities, which we admiditely though of doing, and instead put in long hours and made it work.

## FEEDBACK

Overall this project has turned out to be a challenge and it has taken us quite a long time, forcing us to think out of the box, and go beyond the level of difficulty in the rest of the course.

We feel that the requirements of the project, particularly in exercise 8, exceed greatly the level of theory and practical exercises done throughout the course as some of the tasks were considerably harder than one would expect considering the base level that has been set during the entire semester.